In [ ]:
# This file is to study the fermoionic tensor network (FTN)

# API

In [1]:
from sympy import *
from sympy.physics.quantum import *

In [2]:
from tqdm import tqdm

In [3]:
import numpy as np

## Recursive apply

In [4]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    counter=0
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
        counter+=1
        # print(expr1)
    return expr1

## Variables

In [5]:
g={i:Operator('\gamma_{}'.format(i)) for i in range(8)}
eta={i:Operator('\eta_{}'.format(i)) for i in range(8)}
xi={i:Operator(r'\xi_{}'.format(i)) for i in range(8)}

In [6]:
c={label:Operator(f'c_{label}') for label in ['xi','eta','g']+list(range(4))}

## Rules

### Keeps only external leg

In [7]:
external_leg=[(x,0) for x in list(g.values())+list(eta.values())]

## Keep only non variable, for trace

In [8]:
constant_only=[(x,0) for x in list(xi.values())+list(eta.values())]

In [9]:
constant_gamma_only=[(x,0) for x in g.values()]

### normal order 

In [10]:
selfadjoint=[(Dagger(gi),gi) for gi in list(g.values())+list(eta.values())+list(xi.values())]
normalorder=[gi[i] for gi in [xi,eta,g] for i in range(len(g))]
normalorder2=[(normalorder[j]*normalorder[i],-normalorder[i]*normalorder[j])  if j>i else (normalorder[j]*normalorder[i],1) for i in range(len(normalorder)) for j in range(i,len(normalorder))]

In [11]:
normalorder_C=[element for pair in zip([Dagger(c[i]) for i in range(4)], [c[i] for i in range(4)]) for element in pair]

normalorder_C_dag=[Dagger(c[i]) for i in range(4)]
normalorder_C=[(c[i]) for i in range(4)]

In [12]:
normalorder2_C=[(normalorder_C_dag[i]*normalorder_C_dag[j],-normalorder_C_dag[j]*normalorder_C_dag[i],) if j>i else (normalorder_C_dag[i]*normalorder_C_dag[j],0) for i in range(len(normalorder_C_dag)) for j in range(i,len(normalorder_C_dag))]+[(normalorder_C[i]*normalorder_C[j],-normalorder_C[j]*normalorder_C[i]) if j>i else (normalorder_C[i]*normalorder_C[j],0) for i in range(len(normalorder_C)) for j in range(i,len(normalorder_C))]+[(normalorder_C[i]*normalorder_C_dag[j],-normalorder_C_dag[j]*normalorder_C[i]) if j!=i else (normalorder_C[i]*normalorder_C_dag[j],1-normalorder_C_dag[j]*normalorder_C[i]) for i in range(len(normalorder_C)) for j in range(len(normalorder_C))]

In [13]:
c2g=[(c['xi'],(xi[0]-I*xi[1])/2),
(c['eta'],(eta[0]-I*eta[1])/2),
(c['g'],(g[0]-I*g[1])/2),
(Dagger(c['xi']),(xi[0]+I*xi[1])/2),
(Dagger(c['eta']),(eta[0]+I*eta[1])/2),
(Dagger(c['g']),(g[0]+I*g[1])/2),
(c[0],(g[0]-I*g[1])/2),
(c[1],(g[2]-I*g[3])/2),
(c[2],(g[4]-I*g[5])/2),
(c[3],(g[6]-I*g[7])/2),
]

In [14]:
# rho=(1+Dagger(c['xi'])*Dagger(c['eta']))*(c['xi']*Dagger(c['xi'])*c['eta']*Dagger(c['eta']))*(1+c['eta']*c['xi'])
# rho=(Dagger(c['xi'])*Dagger(c['eta']))*(c['xi']*Dagger(c['xi'])*c['eta']*Dagger(c['eta']))*(c['eta']*c['xi'])
rho=((c['xi']*Dagger(c['xi'])*Dagger(c['eta'])*(c['eta']))+
Dagger(c['xi'])*(c['xi']))*(c['eta']*Dagger(c['eta'])
)


In [15]:
rho_P0=c['xi']*Dagger(c['xi'])*Dagger(c['eta'])*c['eta']
rho_P1=Dagger(c['xi'])*c['xi']*c['eta']*Dagger(c['eta'])


rho=rho_P0+rho_P1+Dagger(c['eta'])*c['xi']+Dagger(c['xi'])*c['eta']

rho_X=c['xi']*Dagger(c['xi']) * c['eta']*Dagger(c['eta']) + Dagger(c['xi'])*c['xi'] * Dagger(c['eta'])*c['eta'] + c['xi']*c['eta'] + Dagger(c['eta'])*Dagger(c['xi'])

### Replace variables

In [16]:
gamma_xi=[(g[idx],xi[idx]) for idx in range(8)]

# Density matrix

In [17]:
def state_to_dm(state,rule=normalorder2):
    return recursiveapply((state*Dagger(state)).subs(selfadjoint).expand(),rule)

# Apply to same hilbert space to recover

In [18]:
def apply_same(op,state,rule=normalorder2):
    return recursiveapply((op*state).expand(),rule)

# Apply to different Hilbert space to contract

In [19]:
p=(1+I*g[0]*eta[0])/2 * (1+I*g[1]*eta[1])/2

In [20]:
def apply_diff(op,state,p=p,rule=normalorder2,external=None):
    '''apply M\otimes I |EPR> to the state and contract.
    the final results will be suppressed to show only external leg'''

    rs= recursiveapply((p*op*state).expand(),rule)
    if external is not None:
        return rs.subs(external)
    else:
        return rs



# Covariance matrix represent

In [21]:
def parity_basis(op,basis):
    assert len(basis)==2, f'len {len(basis)} != 2'
    assert basis[0]!=basis[1], 'two bases should be different'
    return recursiveapply((I*basis[0]*basis[1]*state_to_dm(op)).expand(),normalorder2)
    


In [ ]:
op_p0.subs(gamma_xi)*epr

In [224]:
parity_basis(parity[0],[g[0],g[1]])

1/2 + I*\gamma_0*\gamma_1/2

In [ ]:
state_to_dm(EPR0(eta[0],xi[0]))

In [ ]:
state_to_dm(EPR0(eta[1],xi[1]))

In [ ]:
parity_basis(op_p0.subs(gamma_xi)*epr,[xi[0],xi[1]])

In [ ]:
parity_basis(op_p0.subs(gamma_xi)*epr,[eta[0],eta[1]])

In [ ]:
parity_basis(op_p0.subs(gamma_xi)*epr,[xi[0],eta[0]])

In [ ]:
parity_basis(op_p0.subs(gamma_xi)*epr,[xi[0],eta[1]])

In [ ]:
parity_basis(op_id*epr,[xi[0],xi[1]])

In [ ]:
parity_basis(op_id*epr,[eta[0],eta[1]])

In [ ]:
parity_basis(op_id*epr,[xi[0],eta[1]])

In [ ]:
parity_basis(op_id*epr,[eta[0],xi[1]])

In [ ]:
parity_basis(op_id*epr,[eta[0],xi[0]])

In [ ]:
parity_basis(op_id*epr,[eta[1],xi[1]])

In [ ]:
parity_basis(op_z*epr,[eta[0],xi[0]])

In [ ]:
parity_basis(op_z*epr,[eta[0],xi[1]])

In [ ]:
state_to_dm(op_z*epr)

In [ ]:
state_to_dm(op_id*epr)

In [ ]:
(I*basis[0]*basis[1]*state_to_dm(op)).expand()

In [ ]:
state_to_dm((cosh(a)-sinh(a)*I*xi[0]*xi[1])*epr)

In [ ]:
(cosh(a)-sinh(a)*I*xi[0]*xi[1])*state_to_dm(epr)*(cosh(conjugate(a))-sinh(conjugate(a))*I*xi[0]*xi[1])

In [ ]:
dm_K=recursiveapply(((cosh(a)-sinh(a)*I*xi[0]*xi[1])*state_to_dm(epr)*(cosh(conjugate(a))-sinh(conjugate(a))*I*xi[0]*xi[1])).expand(),normalorder2)

In [ ]:
dm_K

In [ ]:
recursiveapply((I*xi[0]*xi[1]*dm_K).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*xi[0]*eta[0]*dm_K).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*xi[0]*eta[1]*dm_K).expand(),normalorder2).subs(external_leg)

In [ ]:
parity_basis(dm_K,[xi[0],eta[1]])

In [ ]:
parity[0]

In [ ]:
dm_P0P1=recursiveapply(((1+I*xi[0]*xi[1])/2*state_to_dm(epr)*(1-I*xi[0]*xi[1])/2).expand(),normalorder2)

In [ ]:
dm_P1P0=recursiveapply(((1-I*xi[0]*xi[1])/2*state_to_dm(epr)*(1+I*xi[0]*xi[1])/2).expand(),normalorder2)

In [ ]:
dm_P0P1

In [ ]:
recursiveapply((I*xi[0]*xi[1]*dm_P0P1).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*xi[0]*eta[0]*dm_P0P1).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*eta[0]*xi[0]*dm_P1P0).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*eta[1]*xi[1]*dm_P1P0).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*xi[0]*eta[1]*dm_P0P1).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((I*xi[0]*eta[1]*dm_P1P0).expand(),normalorder2).subs(external_leg)

In [ ]:
recursiveapply((parity[1]*parity[0]).expand(),normalorder2)

In [ ]:
EPR0(g[0],g[1])* EPR0(g[2],-g[3])

In [ ]:
(EPR0(g[0],g[1])* EPR0(g[2],-g[3]) + EPR0(g[2],g[3])* EPR0(g[0],-g[1])).expand()

## Class A op

In [ ]:
state_to_dm(epr)

In [ ]:
(1+a* op_plus).subs(gamma_xi)

In [ ]:
rs=I*eta[0]*eta[0]*((1+a* op_plus).subs(gamma_xi) * rho_epr4 * (1+a* op_plus).subs(gamma_xi))

In [ ]:
rs2=recursiveapply(rs.expand(),normalorder2)

In [ ]:
rs2.subs(constant_only)*16

In [ ]:
op_K=(1+a* op_plus).subs(gamma_xi)
op_dm=recursiveapply(( op_K* rho_epr4 * op_K).expand(),normalorder2)

In [ ]:
op_dm

In [ ]:
ij_list=[(i,j) for i in range(8) for j in range(i+1,8)]


In [ ]:
Gamma = zeros(8, 8)
order=[eta[i] for i in range(4)] + [xi[i] for i in range(4)]
ij_list=[(i,j) for i in range(8) for j in range(i+1,8)]
# op_K=(1+a* op_plus).subs(gamma_xi)

op_K=op_u.subs(gamma_xi)
op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)*16
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

In [336]:
cosh(I*a).rewrite(cos)

cos(\alpha)

In [337]:
sinh(I*a).rewrite(sin)

I*sin(\alpha)

In [ ]:
Gamma

In [ ]:
simplify(Gamma@Gamma)

In [ ]:
sqrt(1-(0.761594155955765)**2)

In [ ]:
Gamma

In [ ]:
inspect.getsource(lambdify(a,A,modules='numpy') )

In [ ]:
inspect.getsource(lambdify(a,Gamma[1,6],modules='numpy') )

In [ ]:
Gamma.subs({a:0})

In [ ]:
Gamma.subs({a:1.})

In [ ]:
Gamma2=Gamma@Gamma

In [ ]:
Gamma2.simplify()

In [ ]:
Gamma2

In [ ]:
Gamma2.subs({a:1.})

In [ ]:
A=-Gamma2[0,0].factor()

In [ ]:
A

In [ ]:
import inspect

In [ ]:
inspect.getsource(lambdify(a,Gamma[0,1],modules='numpy') )

In [ ]:
sqrt(A).subs({a:1.5})

In [ ]:
Gamma_0=Gamma*1/sqrt(A)

In [ ]:
Gamma_0_2=Gamma_0@Gamma_0

In [ ]:
Gamma_0_2.simplify()

In [ ]:
Gamma_0_2

In [ ]:
Gamma_0.simplify()

In [ ]:
Gamma_0

In [ ]:
Gamma_0.subs({a:0})

In [ ]:
Gamma_0.subs({a:-.1})

In [ ]:
np.array(Gamma_0.subs({a:0.1})).astype(float)

In [ ]:
Gamma_0.subs({a:80.})

In [ ]:
Gamma_0.subs({a:-80.})

In [ ]:
Gamma_0[1,0].simplify().subs

In [ ]:
rs2.subs(constant_only)

In [ ]:
parity_basis((1+a* op_plus).subs(gamma_xi)*epr4,[eta[0],eta[1]])

In [ ]:
parity_basis((1+a* op_plus).subs(gamma_xi)*epr4,[eta[0],eta[2]])

In [ ]:
parity_basis((1+a* op_plus).subs(gamma_xi)*epr4,[eta[0],eta[3]])

In [ ]:
parity_basis((1+a* op_plus).subs(gamma_xi)*epr4,[eta[0],eta[4]])

# Initialize state

### |0>, |1>, |+>= $\frac{1}{2}$(|0>+|1>), |->= $\frac{1}{2}$(|0>-|1>)

In [210]:
EPR0=lambda s0,s1:(s0-I*s1)/2

In [19]:
rho_EPR0=lambda s0,s1: (1+I*s0*s1)/2

In [212]:
parity={0:EPR0(g[0],g[1]),1:EPR0(g[0],-g[1])}

In [213]:
parity['+']=recursiveapply((1/sqrt(2)*(1+(g[0]+I*g[1])/2)*parity[0]).expand(),normalorder2)
parity['-']=recursiveapply((1/sqrt(2)*(1-(g[0]+I*g[1])/2)*parity[0]).expand(),normalorder2)

In [214]:
state_to_dm(parity[0])

1/2 + I*\gamma_0*\gamma_1/2

In [215]:
state_to_dm(parity[1])

1/2 - I*\gamma_0*\gamma_1/2

In [216]:
state_to_dm(parity['+'])

1/2 + \gamma_0/2

In [217]:
state_to_dm(parity['-'])

1/2 - \gamma_0/2

In [218]:
parity[0]

(\gamma_0 - I*\gamma_1)/2

In [219]:
parity[1]

(\gamma_0 + I*\gamma_1)/2

In [ ]:
parity['+']

In [ ]:
parity['-']

### EPR between $\eta$ and $\xi$

In [ ]:
epr=  EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])

In [220]:
epr4=  EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])*EPR0(eta[2],xi[2])*EPR0(eta[3],xi[3])

In [346]:
# rho_epr4=EPR0(eta[0],xi[0])*EPR0(eta[0],-xi[0])*EPR0(eta[1],xi[1])*EPR0(eta[1],-xi[1])*EPR0(eta[2],xi[2])*EPR0(eta[2],-xi[2])*EPR0(eta[3],xi[3])*EPR0(eta[3],-xi[3])

In [358]:
rho_epr2=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1])

In [23]:
rho_epr4=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1]) * rho_EPR0(eta[2],xi[2]) * rho_EPR0(eta[3],xi[3])

In [ ]:
recursiveapply((EPR0(eta[0],xi[0])*EPR0(eta[0],-xi[0])).expand(),normalorder2)

In [ ]:
recursiveapply((EPR0(eta[0],eta[1])* EPR0(eta[0],-eta[1])).expand(),normalorder2)

# Initialize op

## For Identity

In [ ]:
op_id=1

In [ ]:
apply_same(op_id,parity[0])

In [ ]:
apply_same(op_id,parity[1])

In [ ]:
apply_same(op_id,parity['+'])

In [ ]:
apply_same(op_id,parity['-'])

In [ ]:
(apply_diff(op_id*epr,parity[0],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_id*epr,parity[1],external=external_leg)*(-16*I)).expand()

In [ ]:
(apply_diff(op_id*epr,parity['+'],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_id*epr,parity['-'],external=external_leg)*(16*I)).expand()

In [ ]:
# Gamma=proj*Dagger(proj).subs(selfadjoint)
Gamma=epr*Dagger(epr).subs(selfadjoint)

In [ ]:
Gamma

In [ ]:
recursiveapply((Gamma).expand(),normalorder2)

In [ ]:
recursiveapply((I*eta[0]*xi[0]*Gamma).expand(),normalorder2) 

## For Z, or parity

$P=i \gamma_0 \gamma_1$

$P|0\rangle=|0\rangle$

$P|1\rangle=|1\rangle$

$P |+\rangle=P\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)=|-\rangle$

$P |-\rangle=P\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)=|+\rangle$


In [ ]:
op_z=I*g[0]*g[1]

In [ ]:
apply_same(op_z,parity[0])

In [ ]:
apply_same(op_z,parity[1])

In [ ]:
apply_same(op_z,parity['+'])

In [ ]:
apply_same(op_z,parity['-'])

In [ ]:
(apply_diff(op_z.subs(gamma_xi)*epr,parity[0],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_z.subs(gamma_xi)*epr,parity[1],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_z.subs(gamma_xi)*epr,parity['+'],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_z.subs(gamma_xi)*epr,parity['-'],external=external_leg)*(16*I)).expand()

In [ ]:
Gamma=proj*Dagger(proj).subs(selfadjoint)

In [ ]:
Gamma

In [ ]:
recursiveapply((Gamma).expand(),normalorder2)

## For $P^0=|0 \rangle\langle 0 |= \frac{1+i \gamma_1\gamma_2}{2}$

$P^0|0\rangle=|0\rangle$

$P^0|1\rangle= 0$

$P^0 |+\rangle=P^0\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)=\frac{1}{\sqrt{2}}|0\rangle$

$P^0 |-\rangle=P^0\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)=\frac{1}{\sqrt{2}}|0\rangle$


In [ ]:
op_p0=(1+I*g[0]*g[1])/2

In [ ]:
apply_same(op_p0,parity[0])

In [ ]:
apply_same(op_p0,parity[1])

In [ ]:
apply_same(op_p0,parity['+'])

In [ ]:
apply_same(op_p0,parity['-'])

In [ ]:
(apply_diff(op_p0.subs(gamma_xi)*epr,parity[0],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_p0.subs(gamma_xi)*epr,parity[1],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_p0.subs(gamma_xi)*epr,parity['+'],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_p0.subs(gamma_xi)*epr,parity['-'],external=external_leg)*(16*I)).expand()

In [ ]:
Gamma=proj*Dagger(proj).subs(selfadjoint)

In [ ]:
recursiveapply((Gamma).expand(),normalorder2)

In [ ]:
recursiveapply((Gamma).expand(),normalorder2)

## For $P^1=|1 \rangle\langle 1 |= \frac{1-i \gamma_1\gamma_2}{2}$

$P^1|0\rangle= 0$

$P^1|1\rangle=|1\rangle$


$P^1 |+\rangle=P^1\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)=\frac{1}{\sqrt{2}}|1\rangle$

$P^1 |-\rangle=P^1\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)=-\frac{1}{\sqrt{2}}|1\rangle$


In [ ]:
op_p1=(1-I*g[0]*g[1])/2

In [ ]:
apply_same(op_p1,parity[0])

In [ ]:
state_to_dm(apply_same(op_p1,parity[1]))

In [ ]:
state_to_dm(apply_same(op_p1,parity['+']))

In [ ]:
state_to_dm(apply_same(op_p1,parity['-']))

In [ ]:
(apply_diff(op_p1.subs(gamma_xi)*epr,parity[0],external=external_leg)*(16*I)).expand()

In [ ]:
(apply_diff(op_p1.subs(gamma_xi)*epr,parity[1],external=external_leg)*(-16*I)).expand()

In [ ]:
state_to_dm((apply_diff(op_p1.subs(gamma_xi)*epr,parity['+'],external=external_leg)*(16*I)).expand())

In [ ]:
state_to_dm((apply_diff(op_p1.subs(gamma_xi)*epr,parity['-'],external=external_leg)*(16*I)).expand())

## Test

In [ ]:
recursiveapply((Gamma).expand(),normalorder2)

In [ ]:
recursiveapply((I*eta[0]*eta[0]*Gamma).expand(),normalorder2)

In [ ]:
recursiveapply((I*eta[0]*eta[1]*Gamma).expand(),normalorder2) 

In [ ]:
recursiveapply((I*eta[0]*xi[0]*Gamma).expand(),normalorder2) 

In [ ]:
recursiveapply((I*eta[0]*xi[1]*Gamma).expand(),normalorder2) 

In [ ]:
recursiveapply((I*eta[1]*xi[0]*Gamma).expand(),normalorder2) 

In [ ]:
recursiveapply((I*eta[1]*xi[1]*Gamma).expand(),normalorder2) 

In [ ]:
recursiveapply((I*xi[0]*xi[1]*Gamma).expand(),normalorder2) 

# Project to EPR

In [5]:
alpha=Symbol(r'\alpha',real=True)


In [66]:
n=Symbol('n',real=True)


In [76]:
P={(i,j):Symbol(f'P_{{{i},{j}}}',real=True) for i in range(4) for j in range(i+1,4)}

In [39]:
s1=Symbol('s1',real=True)
s2=Symbol('s2',real=True)

In [20]:
a=Symbol('a',real=False)
b=Symbol('b',real=True)


In [6]:
theta1=Symbol(r'\theta_1',real=True)
theta2=Symbol(r'\theta_2',real=True)


In [178]:
ai=Symbol('a_I',real=True)
ar=Symbol('a_R',real=True)


In [179]:
alpha=ar+I*ai

In [ ]:
cosh(conjugate(alpha)).rewrite(exp).simplify()

In [ ]:
cosh(conjugate(alpha)).rewrite(exp)

In [ ]:
(cosh(ar+I*ai)).rewrite(exp)

In [ ]:
(sinh(alpha)* cosh(conjugate(alpha)) + cosh(alpha)* sinh(conjugate(alpha))).rewrite(exp).expand()

In [ ]:
(sinh(alpha)* sinh(conjugate(alpha)) - cosh(alpha)* cosh(conjugate(alpha))).rewrite(exp).expand()

In [ ]:
(I*sinh(alpha)* cosh(conjugate(alpha)) - I*cosh(alpha)* sinh(conjugate(alpha))).rewrite(exp).expand()

In [ ]:
expand(cosh(a+I*b))

In [ ]:
cosh(a+I*b).simplify()

In [ ]:
ket0=lambda s: (s[0]-I*s[1])/2

In [ ]:
ket0(eta)

In [ ]:
EPR0=lambda s0,s1:(s0-I*s1)/2

In [ ]:
EPR0(eta[0],-xi[0])

In [ ]:
ket0=(g[0]-I*g[1])/2

## For identity

In [ ]:
init=EPR0(g[0],-g[1])

In [ ]:
init

In [ ]:
(p*EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])*EPR0(g[0],g[1]))

In [ ]:
recursiveapply((p*EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])*EPR0(g[0],-g[1])).expand(),normalorder2).subs(external)

In [ ]:
recursiveapply((p*EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])*EPR0(g[0],g[1])).expand(),normalorder2).subs(external)

In [ ]:
recursiveapply(
    (p* proj *EPR0(g[0],g[1])).expand(),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj *EPR0(g[0],a*g[1]) * Dagger(EPR0(g[0],a*g[1]))).expand(),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1-I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1-g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
(1+a**2 + (1-a**2) *I* g[0]*g[1] + 2*a*g[0])

In [ ]:
recursiveapply(
    (p* proj * ((1+a**2 + (1-a**2) *I* g[0]*g[1] + 2*a*g[0])) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

## For Z

In [ ]:
recursiveapply(
    (p* proj * (1-I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1-g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
(1+a**2 + (1-a**2) *I* g[0]*g[1] + 2*a*g[0])

In [ ]:
recursiveapply(
    (p* proj * ((1+a**2 + (1-a**2) *I* g[0]*g[1] + 2*a*g[0])) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

## For P0

In [ ]:
(p*(1+I*eta[0]*eta[1])/2*EPR0(eta[0],xi[0])*EPR0(eta[1],xi[1])*EPR0(g[0],g[1]))

In [ ]:
recursiveapply(
    (p* proj * (1-I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+I*g[0]*g[1])/2 * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1+g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * (1-g[0]) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj * ((1+a**2 + (1-a**2) *I* g[0]*g[1] + 2*a*g[0])) * Dagger(proj)* Dagger(p)).expand().subs(selfadjoint),
    normalorder2
).subs(suppress_eta_g)

In [ ]:
recursiveapply(
    (p* proj *EPR0(g[0],g[1])).expand(),
    normalorder2
)

In [ ]:
recursiveapply(
    (p* proj *EPR0(g[0],-g[1])).expand(),
    normalorder2
)

In [ ]:
recursiveapply(
    (p* proj * g[0]).expand(),
    normalorder2
)

In [ ]:
recursiveapply(
    (p* proj * g[1]).expand(),
    normalorder2
)

In [ ]:
EPR0(g[0],g[1])

# projector for class A

## sigma_x 

In [155]:
c['+']=(c[0]+c[1])/sqrt(2)
c['-']=(c[0]-c[1])/sqrt(2)

In [25]:
recursiveapply((2*Dagger(c['+'])*(c['+'])-1).subs(c2g).expand().subs(selfadjoint),normalorder2)

-I*\gamma_0*\gamma_1/2 - I*\gamma_0*\gamma_3/2 + I*\gamma_1*\gamma_2/2 - I*\gamma_2*\gamma_3/2

In [62]:
recursiveapply((Dagger(c['+'])*(c['+'])*Dagger(c['-'])*(c['-'])).expand(),normalorder2_C)

-Dagger(c_1)*Dagger(c_0)*c_1*c_0

In [64]:
recursiveapply(((c['+'])*Dagger(c['+'])*Dagger(c['-'])*(c['-'])).expand(),normalorder2_C)

Dagger(c_0)*c_0/2 - Dagger(c_0)*c_1/2 + Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_0/2 + Dagger(c_1)*c_1/2

In [27]:
recursiveapply((2*Dagger(c[0])*(c[0])-1).subs(c2g).expand().subs(selfadjoint),normalorder2)

-I*\gamma_0*\gamma_1

In [28]:
recursiveapply((2*Dagger(c[1])*(c[1])-1).subs(c2g).expand().subs(selfadjoint),normalorder2)

-I*\gamma_2*\gamma_3

In [29]:
recursiveapply((Dagger(c['+'])*Dagger(c['-'])).expand(),normalorder2_C)

Dagger(c_1)*Dagger(c_0)

In [30]:
recursiveapply(((c['+'])*Dagger(c['+'])*(c['-'])*Dagger(c['-'])*Dagger(c[0])*Dagger(c[1])).expand(),normalorder2_C)

0

In [31]:
recursiveapply(((c['-'])*Dagger(c['-'])*Dagger(c['+'])*c['+']*Dagger(c[0])*Dagger(c[1])).expand(),normalorder2_C)

0

In [32]:
recursiveapply((Dagger(c['+'])*c['+']).expand(),normalorder2_C)

Dagger(c_0)*c_0/2 + Dagger(c_0)*c_1/2 + Dagger(c_1)*c_0/2 + Dagger(c_1)*c_1/2

In [ ]:
(Dagger(c['-'])*c['-']*Dagger(c[0])*Dagger(c[1])).expand()

In [ ]:
recursiveapply((Dagger(c['-'])*c['-']).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(c['+'])*c[0]*Dagger(c[0])*c[1]*Dagger(c[1])*c['+']*Dagger(c['+'])).expand(),normalorder2_C)

In [45]:
# recover
recursiveapply((Dagger(c['+'])*c['+']-Dagger(c['-'])*c['-']).expand(),normalorder2_C)

Dagger(c_0)*c_1 + Dagger(c_1)*c_0

In [46]:
# commuting
recursiveapply((((Dagger(c['+'])*c['+'])*(Dagger(c['-'])*c['-'])-(Dagger(c['-'])*c['-'])*(Dagger(c['+'])*c['+'])).expand()),normalorder2_C)

0

In [47]:
# commuting
recursiveapply((((Dagger(c['+'])*c['+']-1/2)*(Dagger(c['-'])*c['-']-1/2)-(Dagger(c['-'])*c['-']-1/2)*(Dagger(c['+'])*c['+']-1/2)).expand()),normalorder2_C)

0

In [48]:
# power
recursiveapply(((2*Dagger(c['+'])*c['+']-1)**2).expand(),normalorder2_C)

1

In [49]:
# expand to gamma
recursiveapply((Dagger(c['+'])*c['+']).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 - I*\gamma_0*\gamma_1/4 - I*\gamma_0*\gamma_3/4 + I*\gamma_1*\gamma_2/4 - I*\gamma_2*\gamma_3/4

In [50]:
# expand to gamma
recursiveapply((2*Dagger(c['-'])*c['-']-1).subs(c2g).subs(selfadjoint).expand(),normalorder2)

-I*\gamma_0*\gamma_1/2 + I*\gamma_0*\gamma_3/2 - I*\gamma_1*\gamma_2/2 - I*\gamma_2*\gamma_3/2

In [51]:
# op
op_={(s1,s2):recursiveapply(
    (cosh(a/2)/sqrt(2*cosh(a))+s1*(sinh(a/2)/sqrt(2*cosh(a))*(2*Dagger(c['+'])*c['+']-1)).subs(c2g).subs(selfadjoint).expand())*(cosh(a/2)/sqrt(2*cosh(a))+s2*(sinh(a/2)/sqrt(2*cosh(a))*(2*Dagger(c['-'])*c['-']-1)).subs(c2g).subs(selfadjoint).expand()),normalorder2) for s1 in [-1,1] for s2 in [-1,1]}

In [52]:
op_[1,1]

-sinh(\alpha/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_0*\gamma_1/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_2*\gamma_3/(2*cosh(\alpha)) + cosh(\alpha/2)**2/(2*cosh(\alpha))

In [53]:
prob=recursiveapply(((cosh(s1*a)+sinh(s1*a)*(2*Dagger(c['+'])*c['+']-1)) * (cosh(s2*a)+sinh(s2*a)*(2*Dagger(c['-'])*c['-']-1))).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [54]:
prob

-sinh(\alpha*s_1)*sinh(\alpha*s_2)*\gamma_0*\gamma_1*\gamma_2*\gamma_3 - I*sinh(\alpha*s_1)*cosh(\alpha*s_2)*\gamma_0*\gamma_1/2 - I*sinh(\alpha*s_1)*cosh(\alpha*s_2)*\gamma_0*\gamma_3/2 + I*sinh(\alpha*s_1)*cosh(\alpha*s_2)*\gamma_1*\gamma_2/2 - I*sinh(\alpha*s_1)*cosh(\alpha*s_2)*\gamma_2*\gamma_3/2 - I*sinh(\alpha*s_2)*cosh(\alpha*s_1)*\gamma_0*\gamma_1/2 + I*sinh(\alpha*s_2)*cosh(\alpha*s_1)*\gamma_0*\gamma_3/2 - I*sinh(\alpha*s_2)*cosh(\alpha*s_1)*\gamma_1*\gamma_2/2 - I*sinh(\alpha*s_2)*cosh(\alpha*s_1)*\gamma_2*\gamma_3/2 + cosh(\alpha*s_1)*cosh(\alpha*s_2)

In [55]:
prob_nor=(prob*(1/(2*cosh(a))**2)).subs({cosh(s1*a):cosh(a), cosh(s2*a):cosh(a), sinh(s1*a):s1*sinh(a), sinh(s2*a):s2*sinh(a)}).expand().subs({sinh(a):cosh(a)*tanh(a)})

In [56]:
prob_nor

-s_1*s_2*tanh(\alpha)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/4 - I*s_1*tanh(\alpha)*\gamma_0*\gamma_1/8 - I*s_1*tanh(\alpha)*\gamma_0*\gamma_3/8 + I*s_1*tanh(\alpha)*\gamma_1*\gamma_2/8 - I*s_1*tanh(\alpha)*\gamma_2*\gamma_3/8 - I*s_2*tanh(\alpha)*\gamma_0*\gamma_1/8 + I*s_2*tanh(\alpha)*\gamma_0*\gamma_3/8 - I*s_2*tanh(\alpha)*\gamma_1*\gamma_2/8 - I*s_2*tanh(\alpha)*\gamma_2*\gamma_3/8 + 1/4

In [85]:
g2P={I*g[i]*g[j]:P[i,j] for i in range(4) for j in range(i+1,4)}

In [90]:
collect(prob_nor.subs({tanh(a):n,-g[0]*g[1]*g[2]*g[3]:(P[0,1]*P[2,3]-P[0,2]*P[1,3]+P[0,3]*P[1,2]),**g2P}),list(P.values()))

P_{0,1}*(-n*s1/8 - n*s2/8) + P_{0,3}*(-n*s1/8 + n*s2/8) + P_{1,2}*(n*s1/8 - n*s2/8) + P_{2,3}*(-n*s1/8 - n*s2/8) - n**2*s1*s2*(-P_{0,1}*P_{2,3} + P_{0,2}*P_{1,3} - P_{0,3}*P_{1,2})/4 + 1/4

In [93]:
collect(prob_nor.subs({tanh(a):n,-g[0]*g[1]*g[2]*g[3]:(P[0,1]*P[2,3]-P[0,2]*P[1,3]+P[0,3]*P[1,2]),**g2P}),list(P.values())).subs({s1:1,s2:1})

-P_{0,1}*n/4 - P_{2,3}*n/4 - n**2*(-P_{0,1}*P_{2,3} + P_{0,2}*P_{1,3} - P_{0,3}*P_{1,2})/4 + 1/4

In [94]:
collect(prob_nor.subs({tanh(a):n,-g[0]*g[1]*g[2]*g[3]:(P[0,1]*P[2,3]-P[0,2]*P[1,3]+P[0,3]*P[1,2]),**g2P}),list(P.values())).subs({s1:-1,s2:-1})

P_{0,1}*n/4 + P_{2,3}*n/4 - n**2*(-P_{0,1}*P_{2,3} + P_{0,2}*P_{1,3} - P_{0,3}*P_{1,2})/4 + 1/4

In [96]:
collect(prob_nor.subs({tanh(a):n,-g[0]*g[1]*g[2]*g[3]:(P[0,1]*P[2,3]-P[0,2]*P[1,3]+P[0,3]*P[1,2]),**g2P}),list(P.values())).subs({s1:-1,s2:1})

P_{0,3}*n/4 - P_{1,2}*n/4 + n**2*(-P_{0,1}*P_{2,3} + P_{0,2}*P_{1,3} - P_{0,3}*P_{1,2})/4 + 1/4

In [95]:
collect(prob_nor.subs({tanh(a):n,-g[0]*g[1]*g[2]*g[3]:(P[0,1]*P[2,3]-P[0,2]*P[1,3]+P[0,3]*P[1,2]),**g2P}),list(P.values())).subs({s1:1,s2:-1})

-P_{0,3}*n/4 + P_{1,2}*n/4 + n**2*(-P_{0,1}*P_{2,3} + P_{0,2}*P_{1,3} - P_{0,3}*P_{1,2})/4 + 1/4

In [71]:
prob_nor.subs({tanh(a):n,s1:1,s2:-1})

n**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/4 - I*n*\gamma_0*\gamma_3/4 + I*n*\gamma_1*\gamma_2/4 + 1/4

In [64]:
sum([prob_nor.subs({s1:n1,s2:n2}) for n1 in [-1,1] for n2 in [-1,1]])

1

In [168]:
# op
op_={(s1,s2):recursiveapply(
    (cosh(a/2)/sqrt(2*cosh(a))+s1*(sinh(a/2)/sqrt(2*cosh(a))*(2*Dagger(c['+'])*c['+']-1)).subs(c2g).subs(selfadjoint).expand())*(cosh(a/2)/sqrt(2*cosh(a))+s2*(sinh(a/2)/sqrt(2*cosh(a))*(2*Dagger(c['-'])*c['-']-1)).subs(c2g).subs(selfadjoint).expand()),normalorder2) for s1 in [-1,1] for s2 in [-1,1]}

In [169]:
op_[(1,-1)]

sinh(\alpha/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_0*\gamma_3/(2*cosh(\alpha)) + I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_1*\gamma_2/(2*cosh(\alpha)) + cosh(\alpha/2)**2/(2*cosh(\alpha))

In [170]:
op_[(-1,1)]

sinh(\alpha/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/(2*cosh(\alpha)) + I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_0*\gamma_3/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_1*\gamma_2/(2*cosh(\alpha)) + cosh(\alpha/2)**2/(2*cosh(\alpha))

In [171]:
op_[(1,1)]

-sinh(\alpha/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_0*\gamma_1/(2*cosh(\alpha)) - I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_2*\gamma_3/(2*cosh(\alpha)) + cosh(\alpha/2)**2/(2*cosh(\alpha))

In [172]:
op_[(-1,-1)]

-sinh(\alpha/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/(2*cosh(\alpha)) + I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_0*\gamma_1/(2*cosh(\alpha)) + I*sinh(\alpha/2)*cosh(\alpha/2)*\gamma_2*\gamma_3/(2*cosh(\alpha)) + cosh(\alpha/2)**2/(2*cosh(\alpha))

In [ ]:
recursiveapply((op_plus*op_minus).expand(),normalorder2)

In [ ]:
# op
op_minus=recursiveapply(cosh(-a/2)/sqrt(2*cosh(-a))+(sinh(-a/2)/sqrt(2*cosh(-a))*(2*Dagger(c['-'])*c['-']-1)).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
op_minus

In [165]:
op_pm=(recursiveapply((op_plus*op_minus).expand(),normalorder2).simplify()).expand()

In [ ]:
op_pp=(recursiveapply((op_plus*op_plus).expand(),normalorder2).simplify()).expand()

In [ ]:
recursiveapply((op_minus*op_plus).expand(),normalorder2)

In [ ]:
recursiveapply(((2*Dagger(c['+'])*c['+']-1)**3).expand(),normalorder2_C)

In [ ]:
# normalization
recursiveapply((Dagger(c['+'])*c['+']+Dagger(c['-'])*c['-']).expand(),normalorder2_C)

In [ ]:
# projector
recursiveapply(((Dagger(c['+'])*c['+'])*(Dagger(c['+'])*c['+'])-(Dagger(c['+'])*c['+'])).expand(),normalorder2_C)

In [333]:
# Majorana basis
op_plus=recursiveapply(((Dagger(c['+'])*c['+']).subs(c2g)).expand().subs(selfadjoint),normalorder2)

In [334]:
op_plus

1/2 - I*\gamma_0*\gamma_1/4 - I*\gamma_0*\gamma_3/4 + I*\gamma_1*\gamma_2/4 - I*\gamma_2*\gamma_3/4

In [163]:
# Majorana basis
op_minus=recursiveapply(((Dagger(c['-'])*c['-']).subs(c2g)).expand().subs(selfadjoint),normalorder2)

In [ ]:
recursiveapply((op_plus*op_minus-op_minus*op_plus).expand(),normalorder2)

In [ ]:
((1+(exp(a)-1)*op_plus)*((1+(exp(-a)-1)*op_minus)))

In [ ]:
op_x=recursiveapply(
    ((1+(exp(a)-1)*op_plus)*((1+(exp(-a)-1)*op_minus))).expand(),
    normalorder2).rewrite(sin).expand()

In [ ]:
op_x

In [ ]:
op_x1=-sinh(a)*2*I*g[0]*g[3]+sinh(a)*2*I*g[1]*g[2]-4*cosh(a)+6*(cosh(a)-1)*I*g[0]*g[1]+8*(cosh(a)-1)*g[0]*g[1]*g[2]*g[3]+6*(cosh(a)-1)*I*g[2]*g[3]+5

In [ ]:
op_x1

In [ ]:
(op_x1-op_x).expand()

In [ ]:
(op_x.rewrite(sin)).expand()

In [ ]:
exp(a).rewrite(sin)

In [ ]:
op_x1.subs({a:0})

In [ ]:
(op_x1/(A)**(1/4)).expand().subs({a:50.})

In [ ]:
sqrt(10.)

In [ ]:
0.316227766016838**2

In [ ]:
0.474341649025257**2

In [ ]:
0.632455532033676**2

In [ ]:
0.158113883008419**2

In [ ]:
recursiveapply((2*Dagger(c[0])*c[0]-1).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
op_u=recursiveapply((cos(a/2)+(I*sin(a/2)*(2*Dagger(c[0])*c[0]-1)).subs(c2g).subs(selfadjoint).expand())*(cos(b/2)+(I*sin(b/2)*(2*Dagger(c[1])*c[1]-1)).subs(c2g).subs(selfadjoint).expand())
,normalorder2)

In [ ]:
op_u

## sigma_y

In [ ]:
c['L']=(c[0]-I*c[1])/sqrt(2)
c['R']=(c[0]+I*c[1])/sqrt(2)

In [ ]:
recursiveapply((Dagger(c['R'])*c['R']).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(c['R'])*c['R']*Dagger(c[1])).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(c['R'])*c['R']*Dagger(c[0])*Dagger(c[1])).expand(),normalorder2_C)

In [ ]:
# recover
recursiveapply((Dagger(c['L'])*c['L']-Dagger(c['R'])*c['R']).expand(),normalorder2_C)

In [ ]:
# commuting
recursiveapply((((Dagger(c['L'])*c['L'])*(Dagger(c['R'])*c['R'])-(Dagger(c['R'])*c['R'])*(Dagger(c['L'])*c['L'])).expand()),normalorder2_C)

In [ ]:
# commuting
recursiveapply((((Dagger(c['L'])*c['L']-1/2)*(Dagger(c['R'])*c['R']-1/2)-(Dagger(c['R'])*c['R']-1/2)*(Dagger(c['L'])*c['L']-1/2)).expand()),normalorder2_C)

In [ ]:
# power
recursiveapply(((2*Dagger(c['L'])*c['L']-1)**2).expand(),normalorder2_C)

In [ ]:
# power
recursiveapply(((2*Dagger(c['R'])*c['R']-1)**2).expand(),normalorder2_C)

In [ ]:
# expand to gamma
recursiveapply((Dagger(c['L'])*c['L']).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
# expand to gamma
recursiveapply((Dagger(c['R'])*c['R']).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
# op
op_L=recursiveapply(cosh(a/2)/sqrt(2*cosh(a))+(sinh(a/2)/sqrt(2*cosh(a))*(2*Dagger(c['L'])*c['L']-1)).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
op_L

In [ ]:
# op
op_R=recursiveapply(cosh(-a/2)/sqrt(2*cosh(-a))+(sinh(-a/2)/sqrt(2*cosh(-a))*(2*Dagger(c['R'])*c['R']-1)).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [ ]:
op_R

In [ ]:
recursiveapply((Dagger(c['L'])*c['L']).expand(),normalorder2_C)

In [ ]:
# normalization
recursiveapply((Dagger(c['L'])*c['L']+Dagger(c['R'])*c['R']).expand(),normalorder2_C)

In [ ]:
# projector
recursiveapply(((Dagger(c['L'])*c['L'])*(Dagger(c['L'])*c['L'])-(Dagger(c['L'])*c['L'])).expand(),normalorder2_C)

In [ ]:
# Majorana basis
op_L=recursiveapply(((Dagger(c['L'])*c['L']).subs(c2g)).expand().subs(selfadjoint),normalorder2)

In [ ]:
op_R=recursiveapply(((Dagger(c['R'])*c['R']).subs(c2g)).expand().subs(selfadjoint),normalorder2)

In [ ]:
recursiveapply((Dagger(c['L'])*c['L']-Dagger(c['R'])*c['R']).expand(),normalorder2_C)

# projector for class A

## Verify notes

In [7]:
a=Symbol(r'\alpha',real=True)
s1=Symbol(r's_1',real=True)
s2=Symbol(r's_2',real=True)

In [157]:
K=exp(s1*a*(Dagger(c['+'])*c['+']-1/2))*exp(s2*(Dagger(c['-'])*c['-']-1/2))

In [158]:
sum([(Dagger(K)*K).subs({s1:s10,s2:s20}) for s10 in [-1,1] for s20 in [-1,1]])

0.367879441171442*exp((Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2)*exp(-2*\alpha*(-0.5 + (Dagger(c_0) + Dagger(c_1))*(c_0 + c_1)/2))*exp((Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2) + 0.367879441171442*exp((Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2)*exp(2*\alpha*(-0.5 + (Dagger(c_0) + Dagger(c_1))*(c_0 + c_1)/2))*exp((Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2) + exp(-(-0.5 + (Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2))*exp(-2*\alpha*(-0.5 + (Dagger(c_0) + Dagger(c_1))*(c_0 + c_1)/2))*exp(-(-0.5 + (Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2)) + exp(-(-0.5 + (Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2))*exp(2*\alpha*(-0.5 + (Dagger(c_0) + Dagger(c_1))*(c_0 + c_1)/2))*exp(-(-0.5 + (Dagger(c_0) - Dagger(c_1))*(c_0 - c_1)/2))

# Final contraction

In [39]:
Gamma_={}   

In [40]:
Gamma_[(1,1)]=zeros(8,8)
Gamma_[(1,1)][0,1]=Gamma_[(1,1)][2,3]=tanh(alpha)
Gamma_[(1,1)][4,5]=Gamma_[(1,1)][6,7]=-tanh(alpha)
Gamma_[(1,1)][0,4]=Gamma_[(1,1)][1,5]=Gamma_[(1,1)][2,6]=Gamma_[(1,1)][3,7]=1/cosh(alpha)
Gamma_[(1,1)]=(Gamma_[(1,1)]-Gamma_[(1,1)].T)

In [41]:
Gamma_[(-1,-1)]=zeros(8,8)
Gamma_[(-1,-1)][0,1]=Gamma_[(-1,-1)][2,3]=-tanh(alpha)
Gamma_[(-1,-1)][4,5]=Gamma_[(-1,-1)][6,7]=tanh(alpha)
Gamma_[(-1,-1)][0,4]=Gamma_[(-1,-1)][1,5]=Gamma_[(-1,-1)][2,6]=Gamma_[(-1,-1)][3,7]=1/cosh(alpha)
Gamma_[(-1,-1)]=(Gamma_[(-1,-1)]-Gamma_[(-1,-1)].T)

In [42]:
Gamma_[(-1,1)]=zeros(8,8)
Gamma_[(-1,1)][0,3]=Gamma_[(-1,1)][5,6]=-tanh(alpha)
Gamma_[(-1,1)][1,2]=Gamma_[(-1,1)][4,7]=tanh(alpha)
Gamma_[(-1,1)][0,4]=Gamma_[(-1,1)][1,5]=Gamma_[(-1,1)][2,6]=Gamma_[(-1,1)][3,7]=1/cosh(alpha)
Gamma_[(-1,1)]=(Gamma_[(-1,1)]-Gamma_[(-1,1)].T)

In [43]:
Gamma_[(1,-1)]=zeros(8,8)
Gamma_[(1,-1)][0,3]=Gamma_[(1,-1)][5,6]=tanh(alpha)
Gamma_[(1,-1)][1,2]=Gamma_[(1,-1)][4,7]=-tanh(alpha)
Gamma_[(1,-1)][0,4]=Gamma_[(1,-1)][1,5]=Gamma_[(1,-1)][2,6]=Gamma_[(1,-1)][3,7]=1/cosh(alpha)
Gamma_[(1,-1)]=(Gamma_[(1,-1)]-Gamma_[(1,-1)].T)

In [44]:
Upsilon_=zeros(8,8)
Upsilon_[0,4]=Upsilon_[1,5]=cos(theta1)
Upsilon_[1,4]=sin(theta1)
Upsilon_[0,5]=-sin(theta1)
Upsilon_[2,6]=Upsilon_[3,7]=cos(theta2)
Upsilon_[3,6]=sin(theta2)
Upsilon_[2,7]=-sin(theta2)
Upsilon_=(Upsilon_-Upsilon_.T)

In [45]:
Upsilon_

Matrix([
[             0,              0,              0,              0, cos(\theta_1), -sin(\theta_1),             0,              0],
[             0,              0,              0,              0, sin(\theta_1),  cos(\theta_1),             0,              0],
[             0,              0,              0,              0,             0,              0, cos(\theta_2), -sin(\theta_2)],
[             0,              0,              0,              0,             0,              0, sin(\theta_2),  cos(\theta_2)],
[-cos(\theta_1), -sin(\theta_1),              0,              0,             0,              0,             0,              0],
[ sin(\theta_1), -cos(\theta_1),              0,              0,             0,              0,             0,              0],
[             0,              0, -cos(\theta_2), -sin(\theta_2),             0,              0,             0,              0],
[             0,              0,  sin(\theta_2), -cos(\theta_2),             0,              0,

In [75]:
def contraction_(Gamma,Upsilon,i):
    Gamma_LL=Gamma[:i,:i]
    Gamma_LR=Gamma[:i,i:]
    Gamma_RR=Gamma[i:,i:]
    Upsilon_LL=Upsilon[:i,:i]
    Upsilon_RR=Upsilon[i:,i:]
    Upsilon_RL=Upsilon[i:,:i]
    Mat1=Matrix([[Gamma_LL,0],[0,Gamma_RR]])
    
    pos_mat={}
    for i in range(2):
        for j in range(2):
            mat_zero=zeros(2)
            mat_zero[i,j]=1
            pos_mat[(i,j)]=mat_zero
    one=eye(Gamma_LL.cols)

    mat1=KroneckerProduct(pos_mat[(0,0)],Gamma_LL)+KroneckerProduct(pos_mat[(1,1)],Upsilon_RR)

    mat2=KroneckerProduct(pos_mat[(0,0)],Gamma_LR)+KroneckerProduct(pos_mat[(1,1)],Upsilon_RL)
    mat3=KroneckerProduct(pos_mat[(0,0)],Gamma_RR)+KroneckerProduct(pos_mat[(1,1)],Upsilon_LL)+KroneckerProduct(pos_mat[(0,1)],one)+KroneckerProduct(pos_mat[(1,0)],-one)
    # return mat1, mat2, mat3
    return mat1 + mat2 @ (mat3).inv() @ mat2.T


In [188]:
M=contraction_(Gamma_[(1,1)],Upsilon_,4).simplify()

In [ ]:
M

In [47]:
contraction_(Upsilon_,Gamma_[(1,1)],4).simplify()

Matrix([
[                          0,                tanh(\alpha),                           0,                           0, cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),                          0,                           0],
[              -tanh(\alpha),                           0,                           0,                           0, sin(\theta_1)/cosh(\alpha),  cos(\theta_1)/cosh(\alpha),                          0,                           0],
[                          0,                           0,                           0,                tanh(\alpha),                          0,                           0, cos(\theta_2)/cosh(\alpha), -sin(\theta_2)/cosh(\alpha)],
[                          0,                           0,               -tanh(\alpha),                           0,                          0,                           0, sin(\theta_2)/cosh(\alpha),  cos(\theta_2)/cosh(\alpha)],
[-cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),     

In [48]:
contraction_(Gamma_[(1,1)],Upsilon_,4).simplify()

Matrix([
[                          0,                tanh(\alpha),                           0,                           0, cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),                          0,                           0],
[              -tanh(\alpha),                           0,                           0,                           0, sin(\theta_1)/cosh(\alpha),  cos(\theta_1)/cosh(\alpha),                          0,                           0],
[                          0,                           0,                           0,                tanh(\alpha),                          0,                           0, cos(\theta_2)/cosh(\alpha), -sin(\theta_2)/cosh(\alpha)],
[                          0,                           0,               -tanh(\alpha),                           0,                          0,                           0, sin(\theta_2)/cosh(\alpha),  cos(\theta_2)/cosh(\alpha)],
[-cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),     

In [49]:
contraction_(Upsilon_,Gamma_[(-1,1)],4).simplify()

Matrix([
[                                    0,                                      0, -sin(\theta_1 - \theta_2)*tanh(\alpha), -cos(\theta_1 - \theta_2)*tanh(\alpha), cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),                          0,                           0],
[                                    0,                                      0,  cos(\theta_1 - \theta_2)*tanh(\alpha), -sin(\theta_1 - \theta_2)*tanh(\alpha), sin(\theta_1)/cosh(\alpha),  cos(\theta_1)/cosh(\alpha),                          0,                           0],
[sin(\theta_1 - \theta_2)*tanh(\alpha), -cos(\theta_1 - \theta_2)*tanh(\alpha),                                      0,                                      0,                          0,                           0, cos(\theta_2)/cosh(\alpha), -sin(\theta_2)/cosh(\alpha)],
[cos(\theta_1 - \theta_2)*tanh(\alpha),  sin(\theta_1 - \theta_2)*tanh(\alpha),                                      0,                                      0,       

In [50]:
contraction_(Gamma_[(-1,1)],Upsilon_,4).simplify()

Matrix([
[                          0,                           0,                           0,               -tanh(\alpha),             cos(\theta_1)/cosh(\alpha),           -sin(\theta_1)/cosh(\alpha),                                      0,                                      0],
[                          0,                           0,                tanh(\alpha),                           0,             sin(\theta_1)/cosh(\alpha),            cos(\theta_1)/cosh(\alpha),                                      0,                                      0],
[                          0,               -tanh(\alpha),                           0,                           0,                                      0,                                     0,             cos(\theta_2)/cosh(\alpha),            -sin(\theta_2)/cosh(\alpha)],
[               tanh(\alpha),                           0,                           0,                           0,                                      0,    

In [190]:
M={}

In [191]:
M[(1,1)]=contraction_(Upsilon_,Gamma_[(1,1)],4).simplify()

In [192]:
M[(-1,-1)]=contraction_(Gamma_[(-1,-1)],Upsilon_,4).simplify()

In [193]:
M[(-1,1)]=contraction_(Upsilon_,Gamma_[(-1,1)],4).simplify()

In [197]:
Gamma_[(1,1)]

Matrix([
[              0,    tanh(\alpha),               0,               0, 1/cosh(\alpha),              0,              0,              0],
[  -tanh(\alpha),               0,               0,               0,              0, 1/cosh(\alpha),              0,              0],
[              0,               0,               0,    tanh(\alpha),              0,              0, 1/cosh(\alpha),              0],
[              0,               0,   -tanh(\alpha),               0,              0,              0,              0, 1/cosh(\alpha)],
[-1/cosh(\alpha),               0,               0,               0,              0,  -tanh(\alpha),              0,              0],
[              0, -1/cosh(\alpha),               0,               0,   tanh(\alpha),              0,              0,              0],
[              0,               0, -1/cosh(\alpha),               0,              0,              0,              0,  -tanh(\alpha)],
[              0,               0,               0, -

In [196]:
Gamma_[(-1,1)]

Matrix([
[              0,               0,               0,   -tanh(\alpha), 1/cosh(\alpha),              0,              0,              0],
[              0,               0,    tanh(\alpha),               0,              0, 1/cosh(\alpha),              0,              0],
[              0,   -tanh(\alpha),               0,               0,              0,              0, 1/cosh(\alpha),              0],
[   tanh(\alpha),               0,               0,               0,              0,              0,              0, 1/cosh(\alpha)],
[-1/cosh(\alpha),               0,               0,               0,              0,              0,              0,   tanh(\alpha)],
[              0, -1/cosh(\alpha),               0,               0,              0,              0,  -tanh(\alpha),              0],
[              0,               0, -1/cosh(\alpha),               0,              0,   tanh(\alpha),              0,              0],
[              0,               0,               0, -

In [195]:
Upsilon_

Matrix([
[             0,              0,              0,              0, cos(\theta_1), -sin(\theta_1),             0,              0],
[             0,              0,              0,              0, sin(\theta_1),  cos(\theta_1),             0,              0],
[             0,              0,              0,              0,             0,              0, cos(\theta_2), -sin(\theta_2)],
[             0,              0,              0,              0,             0,              0, sin(\theta_2),  cos(\theta_2)],
[-cos(\theta_1), -sin(\theta_1),              0,              0,             0,              0,             0,              0],
[ sin(\theta_1), -cos(\theta_1),              0,              0,             0,              0,             0,              0],
[             0,              0, -cos(\theta_2), -sin(\theta_2),             0,              0,             0,              0],
[             0,              0,  sin(\theta_2), -cos(\theta_2),             0,              0,

In [194]:
M[(-1,1)]

Matrix([
[                                    0,                                      0, -sin(\theta_1 - \theta_2)*tanh(\alpha), -cos(\theta_1 - \theta_2)*tanh(\alpha), cos(\theta_1)/cosh(\alpha), -sin(\theta_1)/cosh(\alpha),                          0,                           0],
[                                    0,                                      0,  cos(\theta_1 - \theta_2)*tanh(\alpha), -sin(\theta_1 - \theta_2)*tanh(\alpha), sin(\theta_1)/cosh(\alpha),  cos(\theta_1)/cosh(\alpha),                          0,                           0],
[sin(\theta_1 - \theta_2)*tanh(\alpha), -cos(\theta_1 - \theta_2)*tanh(\alpha),                                      0,                                      0,                          0,                           0, cos(\theta_2)/cosh(\alpha), -sin(\theta_2)/cosh(\alpha)],
[cos(\theta_1 - \theta_2)*tanh(\alpha),  sin(\theta_1 - \theta_2)*tanh(\alpha),                                      0,                                      0,       

In [ ]:
M[(1,-1)]=contraction_(Upsilon_,Gamma_[(1,-1)],4).simplify()

In [ ]:
M[(1,-1)]

In [39]:
from importlib import reload

import GTN
reload(GTN)
from GTN import *

In [ ]:
gtn=GTN(L=4,seed=2,op=False,random_init=False)


In [ ]:
np.array(M[(-1,1)].subs({alpha:atanh(0.5),theta1:0.,theta2:0.})).astype(float)

In [ ]:
gtn.op_class_AIII(A=.5,theta1=0,theta2=0,kind=(-1,1))

In [ ]:
gtn.op_class_AIII(A=.5,theta1=0,theta2=0,kind=(-1,1))-np.array(M[(-1,1)].subs({alpha:atanh(0.5),theta1:0.,theta2:0.})).astype(float)

In [ ]:
def test_eq(A,t1,t2):
    for kind in M.keys():
        mat1=gtn.op_class_AIII(A=A,theta1=t1,theta2=t2,kind=kind)
        mat2=np.array(M[kind].subs({alpha:atanh(A),theta1:t1,theta2:t2})).astype(float)
        assert (np.abs(mat1-mat2).max())<1e-12, f'kind ({kind}) failed'
    return True
    

In [ ]:
test_eq(A=1,t1=0,t2=0)
test_eq(A=1,t1=0,t2=0.1)
test_eq(A=1,t1=0.1,t2=0)
test_eq(A=1,t1=0.1,t2=0.1)

In [ ]:
test_eq(A=1,t1=-1,t2=1)
# test_eq(A=1,t1=0,t2=0.1)
# test_eq(A=1,t1=0.1,t2=0)
# test_eq(A=1,t1=0.1,t2=0.1)

In [ ]:
eye(Upsilon_.cols)

In [ ]:
zeros(2)

In [ ]:
Matrix([[Gamma,0],[0,Gamma]])

In [ ]:
Gamma_[(1,1)][:2,:2]

## Check a more generalized case

In [304]:
i,j=0,1

In [305]:
recursiveapply(Dagger(c[i])*c[j]-c[i]*Dagger(c[j]),normalorder2_C)

Dagger(c_0)*c_1 + Dagger(c_1)*c_0

In [306]:
recursiveapply(((Dagger(c[i])*c[j]-c[i]*Dagger(c[j]))**2).expand(),normalorder2_C)

Dagger(c_0)*c_0 + 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 + Dagger(c_1)*c_1

In [307]:
recursiveapply(((Dagger(c[i])*c[j]-c[i]*Dagger(c[j]))**3).expand(),normalorder2_C)

Dagger(c_0)*c_1 + Dagger(c_1)*c_0

In [308]:
recursiveapply(((Dagger(c[i])*c[j]-c[i]*Dagger(c[j]))**4).expand(),normalorder2_C)

Dagger(c_0)*c_0 + 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 + Dagger(c_1)*c_1

In [375]:
i,j=0,1

In [385]:
op_u= (cos(theta1/2)-1) *(Dagger(c[i])*c[j]-c[i]*Dagger(c[j]))**2 + I*sin(theta1/2) *(Dagger(c[i])*c[j]-c[i]*Dagger(c[j])) +1

In [310]:
op_u = cos(theta1/2) + I * sin(theta1/2) *( Dagger(c[i])*c[j]-c[i]*Dagger(c[j]) )

In [386]:
recursiveapply(op_u.expand(),normalorder2_C)

I*sin(\theta_1/2)*Dagger(c_0)*c_1 + I*sin(\theta_1/2)*Dagger(c_1)*c_0 + cos(\theta_1/2)*Dagger(c_0)*c_0 + 2*cos(\theta_1/2)*Dagger(c_1)*Dagger(c_0)*c_1*c_0 + cos(\theta_1/2)*Dagger(c_1)*c_1 + 1 - Dagger(c_0)*c_0 - 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_1

In [387]:
op_u_g=recursiveapply(op_u.subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [388]:
op_u_g

sin(\theta_1/2)*\gamma_0*\gamma_3/2 - sin(\theta_1/2)*\gamma_1*\gamma_2/2 + cos(\theta_1/2)/2 + cos(\theta_1/2)*\gamma_0*\gamma_1*\gamma_2*\gamma_3/2 + 1/2 - \gamma_0*\gamma_1*\gamma_2*\gamma_3/2

In [390]:
recursiveapply((Dagger(op_u)*op_u).expand(),normalorder2_C).rewrite(sin,cos)

cos(\theta_1/2)**2*Dagger(c_0)*c_0 + 2*cos(\theta_1/2)**2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 + cos(\theta_1/2)**2*Dagger(c_1)*c_1 + cos(\theta_1/2 - pi/2)**2*Dagger(c_0)*c_0 + 2*cos(\theta_1/2 - pi/2)**2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 + cos(\theta_1/2 - pi/2)**2*Dagger(c_1)*c_1 + 1 - Dagger(c_0)*c_0 - 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_1

In [391]:
recursiveapply((Dagger(op_u_g)*op_u_g).subs(selfadjoint).expand(),normalorder2)

sin(\theta_1/2)**2/2 + sin(\theta_1/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/2 + cos(\theta_1/2)**2/2 + cos(\theta_1/2)**2*\gamma_0*\gamma_1*\gamma_2*\gamma_3/2 + 1/2 - \gamma_0*\gamma_1*\gamma_2*\gamma_3/2

In [ ]:
parity_basis(op_u_g.subs(gamma_xi)*epr4,[xi[0],xi[1]])

In [322]:
# op_K=op_u_g.subs(gamma_xi)
# op_dm=recursiveapply(( op_K* rho_epr4 * op_K).expand(),normalorder2)

In [331]:
op_u_g

sin(\theta_1/2)*\gamma_0*\gamma_1 + cos(\theta_1/2)

In [330]:
rho_epr4

(1 + I*\eta_0*\xi_0)*(1 + I*\eta_1*\xi_1)*(1 + I*\eta_2*\xi_2)*(1 + I*\eta_3*\xi_3)/16

In [344]:
op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)

In [343]:
recursiveapply(rs.expand(),normalorder2).subs(constant_only)

I*sin(\theta_1/2)*cos(\theta_1/2)/8

In [359]:
op_dm=recursiveapply(( op_K* rho_epr2 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)


In [361]:
op_dm

sin(\theta_1/2)**2/4 + I*sin(\theta_1/2)**2*\xi_0*\eta_0/4 + sin(\theta_1/2)**2*\xi_0*\xi_1*\eta_0*\eta_1/4 + I*sin(\theta_1/2)**2*\xi_1*\eta_1/4 - I*sin(\theta_1/2)*cos(\theta_1/2)*\xi_0*\eta_1/2 + I*sin(\theta_1/2)*cos(\theta_1/2)*\xi_1*\eta_0/2 + cos(\theta_1/2)**2/4 - I*cos(\theta_1/2)**2*\xi_0*\eta_0/4 + cos(\theta_1/2)**2*\xi_0*\xi_1*\eta_0*\eta_1/4 - I*cos(\theta_1/2)**2*\xi_1*\eta_1/4

In [362]:
Gamma = zeros(4, 4)
order=[eta[i] for i in range(2)] + [xi[i] for i in range(2)]
ij_list=[(i,j) for i in range(4) for j in range(i+1,4)]

op_K=op_u_g.subs(gamma_xi)
op_dm=recursiveapply(( op_K* rho_epr2 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)*4
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

100%|██████████| 6/6 [00:10<00:00,  1.73s/it]


In [363]:
Gamma

Matrix([
[             0,              0, cos(\theta_1), -sin(\theta_1)],
[             0,              0, sin(\theta_1),  cos(\theta_1)],
[-cos(\theta_1), -sin(\theta_1),             0,              0],
[ sin(\theta_1), -cos(\theta_1),             0,              0]])

In [392]:
Gamma = zeros(8, 8)
order=[eta[i] for i in range(4)] + [xi[i] for i in range(4)]
ij_list=[(i,j) for i in range(8) for j in range(i+1,8)]

op_K=op_u_g.subs(gamma_xi)
op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)*16
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

100%|██████████| 28/28 [06:09<00:00, 13.20s/it]


In [324]:
order

[\eta_0, \eta_1, \eta_2, \eta_3, \xi_0, \xi_1, \xi_2, \xi_3]

In [393]:
Gamma

Matrix([
[               0,                0,                0,                0, cos(\theta_1/2),                0,               0, -sin(\theta_1/2)],
[               0,                0,                0,                0,               0,  cos(\theta_1/2), sin(\theta_1/2),                0],
[               0,                0,                0,                0,               0, -sin(\theta_1/2), cos(\theta_1/2),                0],
[               0,                0,                0,                0, sin(\theta_1/2),                0,               0,  cos(\theta_1/2)],
[-cos(\theta_1/2),                0,                0, -sin(\theta_1/2),               0,                0,               0,                0],
[               0, -cos(\theta_1/2),  sin(\theta_1/2),                0,               0,                0,               0,                0],
[               0, -sin(\theta_1/2), -cos(\theta_1/2),                0,               0,                0,               0,   

In [394]:
Gamma@Gamma

Matrix([
[-sin(\theta_1/2)**2 - cos(\theta_1/2)**2,                                        0,                                        0,                                        0,                                        0,                                        0,                                        0,                                        0],
[                                       0, -sin(\theta_1/2)**2 - cos(\theta_1/2)**2,                                        0,                                        0,                                        0,                                        0,                                        0,                                        0],
[                                       0,                                        0, -sin(\theta_1/2)**2 - cos(\theta_1/2)**2,                                        0,                                        0,                                        0,                                        0,                     

Here the unitary part can be $\exp(i\theta/2(c_{i,A}^\dag c_{j,A} - c_{i,A} c_{j,A}^\dag))$ and 
$\exp(i\theta/2(c_{i,B}^\dag c_{j,B} - c_{i,B} c_{j,B}^\dag))$
So there are more four complex fermion sites needed

In [8]:
Gamma_={}   

In [9]:
Gamma_[(1,1)]=zeros(16,16)
Gamma_[(-1,-1)]=zeros(16,16)
Gamma_[(-1,1)]=zeros(16,16)
Gamma_[(1,-1)]=zeros(16,16)

In [10]:
Gamma_[(1,1)][0,1]=Gamma_[(1,1)][6,7]=tanh(alpha)
Gamma_[(1,1)][8,9]=Gamma_[(1,1)][14,15]=-tanh(alpha)
Gamma_[(1,1)][0,8]=Gamma_[(1,1)][1,9]=Gamma_[(1,1)][6,14,]=Gamma_[(1,1)][7,15]=1/cosh(alpha)
Gamma_[(1,1)]=(Gamma_[(1,1)]-Gamma_[(1,1)].T)

Gamma_[(-1,-1)][0,1]=Gamma_[(-1,-1)][6,7]=-tanh(alpha)
Gamma_[(-1,-1)][8,9]=Gamma_[(-1,-1)][14,15]=tanh(alpha)
Gamma_[(-1,-1)][0,8]=Gamma_[(-1,-1)][1,9]=Gamma_[(-1,-1)][6,14]=Gamma_[(-1,-1)][7,15]=1/cosh(alpha)
Gamma_[(-1,-1)]=(Gamma_[(-1,-1)]-Gamma_[(-1,-1)].T)

Gamma_[(-1,1)][0,7]=Gamma_[(-1,1)][9,14]=-tanh(alpha)
Gamma_[(-1,1)][1,6]=Gamma_[(-1,1)][8,15]=tanh(alpha)
Gamma_[(-1,1)][0,8]=Gamma_[(-1,1)][1,9]=Gamma_[(-1,1)][6,14]=Gamma_[(-1,1)][7,15]=1/cosh(alpha)
Gamma_[(-1,1)]=(Gamma_[(-1,1)]-Gamma_[(-1,1)].T)

Gamma_[(1,-1)][0,7]=Gamma_[(1,-1)][9,14]=tanh(alpha)
Gamma_[(1,-1)][1,6]=Gamma_[(1,-1)][8,15]=-tanh(alpha)
Gamma_[(1,-1)][0,8]=Gamma_[(1,-1)][1,9]=Gamma_[(1,-1)][6,14]=Gamma_[(1,-1)][7,15]=1/cosh(alpha)

Gamma_[(1,-1)]=(Gamma_[(1,-1)]-Gamma_[(1,-1)].T)

for Gamma in Gamma_.values():
    Gamma[2,10]=Gamma[3,11]=Gamma[4,12]=Gamma[5,13]=1
    Gamma[10,2]=Gamma[11,3]=Gamma[12,4]=Gamma[13,5]=-1


In [11]:

Gamma_[(1,1)]

Matrix([
[              0,    tanh(\alpha),  0,  0,  0,  0,               0,               0, 1/cosh(\alpha),              0, 0, 0, 0, 0,              0,              0],
[  -tanh(\alpha),               0,  0,  0,  0,  0,               0,               0,              0, 1/cosh(\alpha), 0, 0, 0, 0,              0,              0],
[              0,               0,  0,  0,  0,  0,               0,               0,              0,              0, 1, 0, 0, 0,              0,              0],
[              0,               0,  0,  0,  0,  0,               0,               0,              0,              0, 0, 1, 0, 0,              0,              0],
[              0,               0,  0,  0,  0,  0,               0,               0,              0,              0, 0, 0, 1, 0,              0,              0],
[              0,               0,  0,  0,  0,  0,               0,               0,              0,              0, 0, 0, 0, 1,              0,              0],
[              0,  

In [12]:
Gamma_[(1,1)]@Gamma_[(1,1)]

Matrix([
[-tanh(\alpha)**2 - 1/cosh(\alpha)**2,                                    0,  0,  0,  0,  0,                                    0,                                    0,                                    0,                                    0,  0,  0,  0,  0,                                    0,                                    0],
[                                   0, -tanh(\alpha)**2 - 1/cosh(\alpha)**2,  0,  0,  0,  0,                                    0,                                    0,                                    0,                                    0,  0,  0,  0,  0,                                    0,                                    0],
[                                   0,                                    0, -1,  0,  0,  0,                                    0,                                    0,                                    0,                                    0,  0,  0,  0,  0,                                    0,                 

In [13]:
Upsilon_

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [225]:
def get_Upsilon(theta):
    Upsilon_=zeros(8,8)
    Upsilon_[0,4]=Upsilon_[1,5]=Upsilon_[2,6]=Upsilon_[3,7]=cos(theta/2)
    Upsilon_[0,7]=Upsilon_[2,5]=-sin(theta/2)
    Upsilon_[1,6]=Upsilon_[3,4]=sin(theta/2)
    Upsilon_=(Upsilon_-Upsilon_.T)
    return Upsilon_

In [226]:
get_Upsilon(theta1)

Matrix([
[               0,                0,                0,                0, cos(\theta_1/2),                0,               0, -sin(\theta_1/2)],
[               0,                0,                0,                0,               0,  cos(\theta_1/2), sin(\theta_1/2),                0],
[               0,                0,                0,                0,               0, -sin(\theta_1/2), cos(\theta_1/2),                0],
[               0,                0,                0,                0, sin(\theta_1/2),                0,               0,  cos(\theta_1/2)],
[-cos(\theta_1/2),                0,                0, -sin(\theta_1/2),               0,                0,               0,                0],
[               0, -cos(\theta_1/2),  sin(\theta_1/2),                0,               0,                0,               0,                0],
[               0, -sin(\theta_1/2), -cos(\theta_1/2),                0,               0,                0,               0,   

In [227]:
Upsilon_=zeros(16,16)
list1=[0,1,4,5,8,9,12,13]
list2=[2,3,6,7,10,11,14,15]
Upsilon_1=get_Upsilon(theta1)
Upsilon_2=get_Upsilon(theta2)

for i_idx,i in enumerate(list1):
    for j_idx,j in enumerate(list1):
        Upsilon_[i,j] = Upsilon_1[i_idx,j_idx]
for i_idx,i in enumerate(list2):
    for j_idx,j in enumerate(list2):
        Upsilon_[i,j] = Upsilon_2[i_idx,j_idx]

In [17]:
 Upsilon_

Matrix([
[               0,                0,                0,                0,                0,                0,                0,                0, cos(\theta_1/2),                0,               0,                0,               0, -sin(\theta_1/2),               0,                0],
[               0,                0,                0,                0,                0,                0,                0,                0,               0,  cos(\theta_1/2),               0,                0, sin(\theta_1/2),                0,               0,                0],
[               0,                0,                0,                0,                0,                0,                0,                0,               0,                0, cos(\theta_2/2),                0,               0,                0,               0, -sin(\theta_2/2)],
[               0,                0,                0,                0,                0,                0,                0,       

Matrix([
[               0,                0,                0,                0, cos(\theta_1/2),                0,               0, -sin(\theta_1/2)],
[               0,                0,                0,                0,               0,  cos(\theta_1/2), sin(\theta_1/2),                0],
[               0,                0,                0,                0,               0, -sin(\theta_1/2), cos(\theta_1/2),                0],
[               0,                0,                0,                0, sin(\theta_1/2),                0,               0,  cos(\theta_1/2)],
[-cos(\theta_1/2),                0,                0, -sin(\theta_1/2),               0,                0,               0,                0],
[               0, -cos(\theta_1/2),  sin(\theta_1/2),                0,               0,                0,               0,                0],
[               0, -sin(\theta_1/2), -cos(\theta_1/2),                0,               0,                0,               0,   

In [22]:
Upsilon_2

Matrix([
[               0,                0,                0,                0, cos(\theta_2/2),                0,               0, -sin(\theta_2/2)],
[               0,                0,                0,                0,               0,  cos(\theta_2/2), sin(\theta_2/2),                0],
[               0,                0,                0,                0,               0, -sin(\theta_2/2), cos(\theta_2/2),                0],
[               0,                0,                0,                0, sin(\theta_2/2),                0,               0,  cos(\theta_2/2)],
[-cos(\theta_2/2),                0,                0, -sin(\theta_2/2),               0,                0,               0,                0],
[               0, -cos(\theta_2/2),  sin(\theta_2/2),                0,               0,                0,               0,                0],
[               0, -sin(\theta_2/2), -cos(\theta_2/2),                0,               0,                0,               0,   

In [18]:
M={}

In [19]:
M[(1,-1)]=contraction_(Upsilon_,Gamma_[(1,-1)],8).simplify()

In [20]:
M[(1,-1)]

Matrix([
[                                            0,                                            0, -sin(\theta_2/2)*cos(\theta_1/2)*tanh(\alpha),                                             0,                                             0,                                             0,                                             0, cos(\theta_1/2)*cos(\theta_2/2)*tanh(\alpha), cos(\theta_1/2)/cosh(\alpha),                             0,               0,                0,               0, -sin(\theta_1/2),                            0,                             0],
[                                            0,                                            0,                                             0, -sin(\theta_2/2)*cos(\theta_1/2)*tanh(\alpha),                                             0,                                             0, -cos(\theta_1/2)*cos(\theta_2/2)*tanh(\alpha),                                            0,                            0,  cos(\theta_1/2)/c

In [120]:
M[(1,-1)][8:,8:]

Matrix([
[           0,             0, 0, 0, 0, 0,            0, -tanh(\alpha)],
[           0,             0, 0, 0, 0, 0, tanh(\alpha),             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0, -tanh(\alpha), 0, 0, 0, 0,            0,             0],
[tanh(\alpha),             0, 0, 0, 0, 0,            0,             0]])

In [22]:
M[(-1,1)]=contraction_(Upsilon_,Gamma_[(-1,1)],8).simplify()

In [121]:
M[(-1,1)]

Matrix([
[                                              0,                                               0, 2*sin(\theta_2/2)*cos(\theta_1/2)*tanh(\alpha),                                               0,                                              0,                                               0,                                               0, -2*cos(\theta_1/2)*cos(\theta_2/2)*tanh(\alpha),               0,              0, 0, 0, 0, 0,               0,              0],
[                                              0,                                               0,                                              0,  2*sin(\theta_2/2)*cos(\theta_1/2)*tanh(\alpha),                                              0,                                               0,  2*cos(\theta_1/2)*cos(\theta_2/2)*tanh(\alpha),                                               0,               0,              0, 0, 0, 0, 0,               0,              0],
[-2*sin(\theta_2/2)*cos(\theta_1/2)*tanh(\alpha),  

In [118]:
M[(-1,1)][:8,8:]

Matrix([
[cos(\theta_1/2)/cosh(\alpha),                             0,               0,                0,               0, -sin(\theta_1/2),                            0,                             0],
[                           0,  cos(\theta_1/2)/cosh(\alpha),               0,                0, sin(\theta_1/2),                0,                            0,                             0],
[                           0,                             0, cos(\theta_2/2),                0,               0,                0,                            0, -sin(\theta_2/2)/cosh(\alpha)],
[                           0,                             0,               0,  cos(\theta_2/2),               0,                0, sin(\theta_2/2)/cosh(\alpha),                             0],
[                           0, -sin(\theta_1/2)/cosh(\alpha),               0,                0, cos(\theta_1/2),                0,                            0,                             0],
[sin(\theta_1/2)/cosh

In [25]:
M[(1,1)]=contraction_(Upsilon_,Gamma_[(1,1)],8).simplify()

In [26]:
M[(1,1)]

Matrix([
[                               0, cos(\theta_1/2)**2*tanh(\alpha),                                0,                               0,    -sin(\theta_1)*tanh(\alpha)/2,                               0,                                0,                               0, cos(\theta_1/2)/cosh(\alpha),                             0,               0,                0,               0, -sin(\theta_1/2),                            0,                             0],
[-cos(\theta_1/2)**2*tanh(\alpha),                               0,                                0,                               0,                                0,   -sin(\theta_1)*tanh(\alpha)/2,                                0,                               0,                            0,  cos(\theta_1/2)/cosh(\alpha),               0,                0, sin(\theta_1/2),                0,                            0,                             0],
[                               0,                               0,

In [27]:
M[(1,1)][8:,8:]

Matrix([
[           0, -tanh(\alpha), 0, 0, 0, 0,            0,             0],
[tanh(\alpha),             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0,             0],
[           0,             0, 0, 0, 0, 0,            0, -tanh(\alpha)],
[           0,             0, 0, 0, 0, 0, tanh(\alpha),             0]])

In [28]:
M[(-1,-1)]=contraction_(Upsilon_,Gamma_[(-1,-1)],8).simplify()

In [29]:
M[(-1,-1)]

Matrix([
[                              0, -cos(\theta_1/2)**2*tanh(\alpha),                               0,                                0,    sin(\theta_1)*tanh(\alpha)/2,                                0,                               0,                                0, cos(\theta_1/2)/cosh(\alpha),                             0,               0,                0,               0, -sin(\theta_1/2),                            0,                             0],
[cos(\theta_1/2)**2*tanh(\alpha),                                0,                               0,                                0,                               0,     sin(\theta_1)*tanh(\alpha)/2,                               0,                                0,                            0,  cos(\theta_1/2)/cosh(\alpha),               0,                0, sin(\theta_1/2),                0,                            0,                             0],
[                              0,                                0,

In [30]:
M[(-1,-1)][8:,8:]

Matrix([
[            0, tanh(\alpha), 0, 0, 0, 0,             0,            0],
[-tanh(\alpha),            0, 0, 0, 0, 0,             0,            0],
[            0,            0, 0, 0, 0, 0,             0,            0],
[            0,            0, 0, 0, 0, 0,             0,            0],
[            0,            0, 0, 0, 0, 0,             0,            0],
[            0,            0, 0, 0, 0, 0,             0,            0],
[            0,            0, 0, 0, 0, 0,             0, tanh(\alpha)],
[            0,            0, 0, 0, 0, 0, -tanh(\alpha),            0]])

In [208]:
from importlib import reload

import GTN
reload(GTN)
from GTN import *

In [209]:
gtn=GTN(L=8,seed=2,op=False,random_init=False)


In [217]:
kind=(-1,-1)
A_0=0.5
theta1_0=1.
theta2_0=0.5
M_a=M[kind].subs({alpha:atanh(A_0),theta1:theta1_0,theta2:theta2_0})

In [218]:
M_n=gtn.op_class_AIII_unitary(A=A_0,theta1=theta1_0,theta2=theta2_0,kind=kind)

In [219]:
M_a-M_n

Matrix([
[0, 0,                    0,                     0,                    0,                     0,                     0,                     0,                    0,                     0, 0, 0, 0, 0,                    0,                     0],
[0, 0,                    0,                     0,                    0,                     0,                     0,                     0,                    0,                     0, 0, 0, 0, 0,                    0,                     0],
[0, 0,                    0,                     0,                    0,                     0,                     0,                     0,                    0,                     0, 0, 0, 0, 0,                    0, -2.77555756156289e-17],
[0, 0,                    0,                     0,                    0,                     0,                     0,                     0,                    0,                     0, 0, 0, 0, 0, 2.77555756156289e-17,                     0],
[0, 0, 

In [220]:
(M_a)[:8,:8]

Matrix([
[                 0, -0.385075576467035,                  0,                   0, 0.210367746201974,                  0,                  0,                  0],
[ 0.385075576467035,                  0,                  0,                   0,                 0,  0.210367746201974,                  0,                  0],
[                 0,                  0,                  0, -0.0306043595274068,                 0,                  0, -0.119856384651051,                  0],
[                 0,                  0, 0.0306043595274068,                   0,                 0,                  0,                  0, -0.119856384651051],
[-0.210367746201974,                  0,                  0,                   0,                 0, -0.114924423532965,                  0,                  0],
[                 0, -0.210367746201974,                  0,                   0, 0.114924423532965,                  0,                  0,                  0],
[                 0

In [221]:
np.round(M_n[:8,:8],2)

array([[ 0.  , -0.39,  0.  ,  0.  ,  0.21,  0.  ,  0.  ,  0.  ],
       [ 0.39,  0.  ,  0.  ,  0.  ,  0.  ,  0.21,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.03,  0.  ,  0.  , -0.12,  0.  ],
       [ 0.  ,  0.  ,  0.03,  0.  ,  0.  ,  0.  ,  0.  , -0.12],
       [-0.21,  0.  ,  0.  ,  0.  ,  0.  , -0.11,  0.  ,  0.  ],
       [ 0.  , -0.21,  0.  ,  0.  ,  0.11,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.12,  0.  ,  0.  ,  0.  ,  0.  , -0.47],
       [ 0.  ,  0.  ,  0.  ,  0.12,  0.  ,  0.  ,  0.47,  0.  ]])

In [222]:
(M_a-M_n)[:8,:8]

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]])

In [216]:
print(np.abs(M_a-M_n).sum())

7.77156117237610e-16


In [161]:
def check_diff(kind=(1,-1),seed=1):
    rng= np.random.default_rng(seed)
    A_0=rng.uniform(0,1)
    theta1_0=rng.uniform(0,1)*2*np.pi
    theta2_0=rng.uniform(0,1)*2*np.pi
    M_a=M[kind].subs({alpha:atanh(A_0),theta1:theta1_0,theta2:theta2_0})
    M_n=gtn.op_class_AIII_unitary(A=A_0,theta1=theta1_0,theta2=theta2_0,kind=kind)
    if np.abs(M_a-M_n).sum()>1e-10:
        print(A_0,theta1_0,theta2_0)
    else:
        pass
        # print('pass')


In [164]:
for s in range(100):
    check_diff(kind=(1,-1),seed=s)

In [165]:
for s in range(100):
    check_diff(kind=(-1,1),seed=s)

In [223]:
for s in range(100):
    check_diff(kind=(1,1),seed=s)

In [224]:
for s in range(100):
    check_diff(kind=(-1,-1),seed=s)

# Wave function -> Covariance

In [16]:
rho_EPR=lambda s0,s1,p: (1+I*s0*s1*p)/2

In [17]:
rho_epr=rho_EPR(g[0],g[2],1)*rho_EPR(g[1],g[3],1)

In [37]:
cosh(ai+I*ar).rewrite(exp)

exp(-a_I - I*a_R)/2 + exp(a_I + I*a_R)/2

In [29]:
state=(cosh(ai+I*ar)+sinh(ai+I*ar)*I*g[0]*g[1]) * rho_epr*(cosh((ai-I*ar))+sinh((ai-I*ar))*I*g[0]*g[1])

In [38]:
recursiveapply((state.rewrite(exp).expand()),normalorder2)

exp(2*a_I)/8 + I*exp(2*a_I)*\gamma_0*\gamma_1/8 + exp(2*a_I)*\gamma_0*\gamma_1*\gamma_2*\gamma_3/8 - I*exp(2*a_I)*\gamma_2*\gamma_3/8 + I*exp(2*I*a_R)*\gamma_0*\gamma_2/8 - exp(2*I*a_R)*\gamma_0*\gamma_3/8 + exp(2*I*a_R)*\gamma_1*\gamma_2/8 + I*exp(2*I*a_R)*\gamma_1*\gamma_3/8 + I*exp(-2*I*a_R)*\gamma_0*\gamma_2/8 + exp(-2*I*a_R)*\gamma_0*\gamma_3/8 - exp(-2*I*a_R)*\gamma_1*\gamma_2/8 + I*exp(-2*I*a_R)*\gamma_1*\gamma_3/8 + exp(-2*a_I)/8 - I*exp(-2*a_I)*\gamma_0*\gamma_1/8 + exp(-2*a_I)*\gamma_0*\gamma_1*\gamma_2*\gamma_3/8 + I*exp(-2*a_I)*\gamma_2*\gamma_3/8

In [234]:
c['0+']=(cos(theta1)+sin(theta1)*exp(-I*alpha)*c[0])/sqrt(2)
# c['0-']=(1-I*Dagger(c[0]))/sqrt(2)

In [235]:
recursiveapply(((Dagger(c[0])*(c[0]))).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 - I*\gamma_0*\gamma_1/2

In [236]:
recursiveapply((((c[0])*Dagger(c[0]))).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 + I*\gamma_0*\gamma_1/2

In [237]:
rho_0p=recursiveapply(((Dagger(c['0+'])*(c[0])*Dagger(c[0])*(c['0+']))).subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [240]:
rho_0p

exp(I*\alpha)*sin(\theta_1)*cos(\theta_1)*\gamma_0/4 + I*exp(I*\alpha)*sin(\theta_1)*cos(\theta_1)*\gamma_1/4 + sin(\theta_1)**2/4 - I*sin(\theta_1)**2*\gamma_0*\gamma_1/4 + cos(\theta_1)**2/4 + I*cos(\theta_1)**2*\gamma_0*\gamma_1/4 + exp(-I*\alpha)*sin(\theta_1)*cos(\theta_1)*\gamma_0/4 - I*exp(-I*\alpha)*sin(\theta_1)*cos(\theta_1)*\gamma_1/4

In [231]:
recursiveapply((((c['0-'])*(c[0])*Dagger(c[0])*Dagger(c['0-']))).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 + \gamma_1/2

In [232]:
Gamma = zeros(2, 2)
# order=[eta[i] for i in range(4)] + [xi[i] for i in range(4)]
order=g
ij_list=[(i,j) for i in range(2) for j in range(i+1,2)]
# op_K=(1+a* op_plus).subs(gamma_xi)

op_dm=recursiveapply(( rho_0p* rho_epr * Dagger(rho_0p)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_gamma_only)*2
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

100%|██████████| 1/1 [00:00<00:00,  9.53it/s]


In [233]:
Gamma

Matrix([
[0, 0],
[0, 0]])

# Check initial state in fermionic basis

In [107]:
c2g=[(c[0],(g[0]-I*g[1])/2),(c[1],(g[2]-I*g[3])/2)]

In [96]:
# c2g=[(c[0],(g[0]-I*g[1])/2),(c[1],(g[3]-I*g[2])/2)]

In [90]:
g2c=[(g[0],c[0]+Dagger(c[0])),(g[1],-I*(c[0]-Dagger(c[0]))),(g[2],c[1]+Dagger(c[1])),(g[3],-I*(c[1]-Dagger(c[1])))]

In [128]:
# bell_0011=(((1+Dagger(c[0])*Dagger(c[1])) * (c[0]*Dagger(c[0])*c[1]*Dagger(c[1])) * (1+(c[1])*(c[0]))))/2
# bell_0011=(((1+I*Dagger(c[0])*(c[1])) * (c[0]*Dagger(c[0])*Dagger(c[1])*(c[1])) * (1-I*Dagger(c[1])*(c[0]))))/2
bell_0110=(((I*Dagger(c[0])+ Dagger(c[1])) * (c[0]*Dagger(c[0])*c[1]*Dagger(c[1])) * (-I*(c[0])+ (c[1]))))/2

In [114]:
bell_0011

(1 + I*Dagger(c_0)*c_1)*c_0*Dagger(c_0)*Dagger(c_1)*c_1*(1 - I*Dagger(c_1)*c_0)/2

In [129]:
bell_0110

(I*Dagger(c_0) + Dagger(c_1))*c_0*Dagger(c_0)*c_1*Dagger(c_1)*(-I*c_0 + c_1)/2

In [130]:
# bell_0011_c=recursiveapply(bell_0011.expand(),normalorder2_C)
bell_0110_c=recursiveapply(bell_0110.expand(),normalorder2_C)

In [121]:
# bell_0011_c

Dagger(c_0)*c_0/2 + I*Dagger(c_0)*c_1/2 + Dagger(c_1)*Dagger(c_0)*c_1*c_0 - I*Dagger(c_1)*c_0/2 + Dagger(c_1)*c_1/2

In [131]:
bell_0110_c

Dagger(c_0)*c_0/2 + I*Dagger(c_0)*c_1/2 + Dagger(c_1)*Dagger(c_0)*c_1*c_0 - I*Dagger(c_1)*c_0/2 + Dagger(c_1)*c_1/2

In [117]:
recursiveapply((bell_0011_c.subs(c2g)).expand().subs(selfadjoint),normalorder2)


1/4 + \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 + I*\gamma_0*\gamma_2/4 + I*\gamma_1*\gamma_3/4

In [132]:
recursiveapply((bell_0110_c.subs(c2g)).expand().subs(selfadjoint),normalorder2)

1/4 + \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 + I*\gamma_0*\gamma_2/4 + I*\gamma_1*\gamma_3/4

In [124]:
recursiveapply((((1+I*g[0]*g[2])/2 * (1+I*g[1]*g[3])/2).expand()),normalorder2)

1/4 + \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 + I*\gamma_0*\gamma_2/4 + I*\gamma_1*\gamma_3/4

In [89]:
recursiveapply(((1+I*g[1]*g[3])/2 * (1+I*g[0]*g[2])/2 ).expand(),normalorder2)

1/4 + \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 + I*\gamma_0*\gamma_2/4 + I*\gamma_1*\gamma_3/4

In [42]:
((1+I*g[0]*g[1])/2 * (1+I*g[2]*g[3])/2).expand()

1/4 + I*\gamma_0*\gamma_1/4 - \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 + I*\gamma_2*\gamma_3/4

In [93]:
recursiveapply(recursiveapply((((1+I*g[0]*g[2])/2 * (1+I*g[1]*g[3])/2).expand()),normalorder2).subs(g2c).expand(),normalorder2_C)

Dagger(c_0)*c_0/2 + I*Dagger(c_0)*c_1/2 + Dagger(c_1)*Dagger(c_0)*c_1*c_0 - I*Dagger(c_1)*c_0/2 + Dagger(c_1)*c_1/2

# 2+1D class DIII -> 2D class D

In [15]:
(I*Dagger(c[1])*Dagger(c[0]) +I * c[1]*c[0]).subs(c2g).subs(selfadjoint).expand()

I*\gamma_2*\gamma_0/2 - I*\gamma_3*\gamma_1/2

In [16]:
(Dagger(c[1])*Dagger(c[0]) - c[1]*c[0]).subs(c2g).subs(selfadjoint).expand()

I*\gamma_2*\gamma_1/2 + I*\gamma_3*\gamma_0/2

In [17]:
recursiveapply((Dagger(c[1])*c[0] + Dagger(c[0])*c[1]).subs(c2g).subs(selfadjoint).expand(),normalorder2)

-I*\gamma_0*\gamma_3/2 + I*\gamma_1*\gamma_2/2

In [18]:
recursiveapply((Dagger(c[1])*c[0] + Dagger(c[0])*c[1]).subs(c2g).subs(selfadjoint).expand(),normalorder2)

-I*\gamma_0*\gamma_3/2 + I*\gamma_1*\gamma_2/2

In [19]:
recursiveapply((Dagger(c[0])*c[0] ).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 - I*\gamma_0*\gamma_1/2

# 1D Kitaev chain

In [45]:
P11=c[0]*Dagger(c[0])/2+(c[1]*Dagger(c[0])+c[0]*Dagger(c[1]))/4

In [46]:
P22=Dagger(c[0])*(c[0])/2-(Dagger(c[1])*(c[0])+Dagger(c[0])*(c[1]))/4

In [47]:
# -(c[0]*c[1]-c[1]*c[0])/4
P12=- c[0] * c[1]/2

In [48]:
P21=- Dagger(c[1]) * Dagger(c[0])/2

In [56]:
Ptotal=recursiveapply(P11+P22+P12+P21,normalorder2_C)

In [58]:
Ptotal

1/2 - Dagger(c_0)*c_1/2 - Dagger(c_1)*Dagger(c_0)/2 - Dagger(c_1)*c_0/2 + c_1*c_0/2

In [ ]:
-(Dagger(c[1])*Dagger(c[0])-Dagger(c[0])*Dagger(c[1]))

In [30]:
recursiveapply((c[0]*Dagger(c[0])/2+(c[1]*Dagger(c[0])+c[0]*Dagger(c[1]))/4).subs(c2g).subs(selfadjoint).expand(), normalorder2)

1/4 + I*\gamma_0*\gamma_1/4 + I*\gamma_0*\gamma_3/8 - I*\gamma_1*\gamma_2/8

In [32]:
recursiveapply((
    Dagger(c[0])*(c[0])/2-(Dagger(c[1])*(c[0])+Dagger(c[0])*(c[1]))/4
).subs(c2g).subs(selfadjoint).expand(), normalorder2)

1/4 - I*\gamma_0*\gamma_1/4 + I*\gamma_0*\gamma_3/8 - I*\gamma_1*\gamma_2/8

In [42]:
recursiveapply((
    # -(c[0]*c[1]-c[1]*c[0])/4
    - c[0] * c[1]/2
).subs(c2g).subs(selfadjoint).expand(), normalorder2)

-\gamma_0*\gamma_2/8 + I*\gamma_0*\gamma_3/8 + I*\gamma_1*\gamma_2/8 + \gamma_1*\gamma_3/8

In [44]:
recursiveapply((
    - Dagger(c[1]) * Dagger(c[0])/2
).subs(c2g).subs(selfadjoint).expand(), normalorder2)

\gamma_0*\gamma_2/8 + I*\gamma_0*\gamma_3/8 + I*\gamma_1*\gamma_2/8 - \gamma_1*\gamma_3/8

In [57]:
recursiveapply((
    Ptotal
).subs(c2g).subs(selfadjoint).expand(), normalorder2)

1/2 + I*\gamma_0*\gamma_3/2

In [23]:
recursiveapply((c[0]*Dagger(c[0])).subs(c2g).subs(selfadjoint).expand(),normalorder2)

1/2 + I*\gamma_0*\gamma_1/2

In [ ]:
Dagger(c[0])*(c[0])/2-(Dagger(c[1])*(c[0])+Dagger(c[0])*(c[1]))/4

-(Dagger(c_0)*c_1 + Dagger(c_1)*c_0)/4 + Dagger(c_0)*c_0/2

# 1D SSH

(c_0 - c_1)*(Dagger(c_0) - Dagger(c_1))

In [72]:
recursiveapply(((Dagger(c[0])-Dagger(c[1]) ) * ((c[0])-(c[1]) )* Dagger(((c[0])-(c[1]) )) * ((c[0])+(c[1]) )* Dagger(((c[0])+(c[1]) )) * (c[0]-c[1])/4).expand(),normalorder2_C)

Dagger(c_0)*c_0 - Dagger(c_0)*c_1 + 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_0 + Dagger(c_1)*c_1

In [64]:
recursiveapply(((Dagger(c[0])-Dagger(c[1]) ) * c[0]*Dagger(c[0])*c[1]*Dagger(c[1]) * (c[0]-c[1])).expand(),normalorder2_C)

Dagger(c_0)*c_0 - Dagger(c_0)*c_1 + 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_0 + Dagger(c_1)*c_1

In [65]:
recursiveapply(((Dagger(c[0])-Dagger(c[1]) ) * c[0]*Dagger(c[0])*c[1]*Dagger(c[1]) * (c[0]-c[1])).subs({c[0]:Dagger(c[0]),c[1]:-Dagger(c[1])},simultaneous=True).expand(),normalorder2_C)

Dagger(c_0)*c_0 - Dagger(c_0)*c_1 + 2*Dagger(c_1)*Dagger(c_0)*c_1*c_0 - Dagger(c_1)*c_0 + Dagger(c_1)*c_1

In [ ]:
subs

## Charge transfer

In [139]:
c2g

[(c_xi, (\xi_0 - I*\xi_1)/2),
 (c_eta, (\eta_0 - I*\eta_1)/2),
 (c_g, (\gamma_0 - I*\gamma_1)/2),
 (Dagger(c_xi), (\xi_0 + I*\xi_1)/2),
 (Dagger(c_eta), (\eta_0 + I*\eta_1)/2),
 (Dagger(c_g), (\gamma_0 + I*\gamma_1)/2),
 (c_0, (\gamma_0 - I*\gamma_1)/2),
 (c_1, (\gamma_2 - I*\gamma_3)/2)]

In [138]:
c

{'xi': c_xi,
 'eta': c_eta,
 'g': c_g,
 0: c_0,
 1: c_1,
 2: c_2,
 3: c_3,
 '+': sqrt(2)*(c_0 + c_1)/2,
 '-': sqrt(2)*(c_0 - c_1)/2}

In [108]:
c['+t'] = (c[0]+c[1])/sqrt(2)
c['-t'] = (c[0]-c[1])/sqrt(2)

c['+b'] = (c[2]+c[3])/sqrt(2)
c['-b'] = (c[2]-c[3])/sqrt(2)

# c['bA']


In [109]:
# op= Dagger(c[0])*c[1]
# op= Dagger(c[1])*c[0]

op = Dagger(c['-t']) * c['+t']* Dagger(c['+t'])
# Dagger(c['-t']) 
# * (c[2])


# op_m = Dagger(c['-t'])* c['-t'] *  c['+t'] * Dagger(c['+t'])

# op_p = Dagger(c['+t'])* c['+t'] *  c['-t'] * Dagger(c['-t'])

# * c['+t'] * Dagger(c['+t'])

# op = Dagger(c['+t']) * c['+t'] * c['-t'] * Dagger(c['-t'])

# op = Dagger(c['-t']) * c['+t']*Dagger(c['+t']) * c['+b']*Dagger(c['+b']) * (c['-b'])

In [128]:
# op= Dagger(c[0]-c[1])*(c[0]-c[1])/2

In [110]:
op_g = recursiveapply(op.subs(c2g).subs(selfadjoint).expand(),normalorder2)

In [111]:
op_g

sqrt(2)*\gamma_0/8 - sqrt(2)*I*\gamma_0*\gamma_1*\gamma_2/8 + sqrt(2)*\gamma_0*\gamma_1*\gamma_3/8 + sqrt(2)*I*\gamma_0*\gamma_2*\gamma_3/8 + sqrt(2)*I*\gamma_1/8 - sqrt(2)*\gamma_1*\gamma_2*\gamma_3/8 - sqrt(2)*\gamma_2/8 - sqrt(2)*I*\gamma_3/8

In [112]:
op_K=op_g.subs(gamma_xi)

In [113]:
op_K

sqrt(2)*\xi_0/8 - sqrt(2)*I*\xi_0*\xi_1*\xi_2/8 + sqrt(2)*\xi_0*\xi_1*\xi_3/8 + sqrt(2)*I*\xi_0*\xi_2*\xi_3/8 + sqrt(2)*I*\xi_1/8 - sqrt(2)*\xi_1*\xi_2*\xi_3/8 - sqrt(2)*\xi_2/8 - sqrt(2)*I*\xi_3/8

In [44]:
rho_epr2=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1]) 

In [114]:
rho_epr4=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1]) * rho_EPR0(eta[2],xi[2]) * rho_EPR0(eta[3],xi[3])

In [115]:
rho_epr6=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1]) * rho_EPR0(eta[2],xi[2]) * rho_EPR0(eta[3],xi[3]) * rho_EPR0(eta[4],xi[4]) * rho_EPR0(eta[5],xi[5]) 

In [232]:
rho_epr8=rho_EPR0(eta[0],xi[0]) * rho_EPR0(eta[1],xi[1]) * rho_EPR0(eta[2],xi[2]) * rho_EPR0(eta[3],xi[3]) * rho_EPR0(eta[4],xi[4]) * rho_EPR0(eta[5],xi[5]) * rho_EPR0(eta[6],xi[6]) * rho_EPR0(eta[7],xi[7])

In [61]:
op_dm=recursiveapply(( op_K* rho_epr2 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)

In [ ]:
op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)

In [ ]:
op_dm=recursiveapply(( op_K* rho_epr6 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)

In [ ]:
op_dm

1/8 + I*\eta_0*\eta_1/8 + I*\xi_0*\xi_1/8 - \xi_0*\xi_1*\eta_0*\eta_1/8

In [63]:
def get_Gamma(op_dm,dim):
    Gamma = zeros(2*dim, 2*dim)
    order=[eta[i] for i in range(dim)] + [xi[i] for i in range(dim)]
    ij_list=[(i,j) for i in range(2*dim) for j in range(i+1,2*dim)]

    # op_K=op_u_g.subs(gamma_xi)
    # op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
    for i,j in tqdm(ij_list):
            rs=I*order[i]*order[j]*( op_dm)
            matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)*16
            matel=matel.simplify()
            Gamma[i,j]=matel
            Gamma[j,i]=-matel
    return Gamma

In [ ]:
Gamma=get_Gamma(op_dm,4)

100%|██████████| 6/6 [00:06<00:00,  1.01s/it]


In [68]:
Gamma_dag=Gamma_dag/2

In [69]:
Gamma_dag@Gamma_dag

Matrix([
[-1,  0,  0,  0],
[ 0, -1,  0,  0],
[ 0,  0, -1,  0],
[ 0,  0,  0, -1]])

In [64]:
Gamma_=get_Gamma(op_dm,2)

100%|██████████| 6/6 [00:06<00:00,  1.02s/it]


In [70]:
Gamma_/=2

In [71]:
Gamma_@Gamma_

Matrix([
[-1,  0,  0,  0],
[ 0, -1,  0,  0],
[ 0,  0, -1,  0],
[ 0,  0,  0, -1]])

In [80]:
Gamma_dag

Matrix([
[0, -1, 0,  0],
[1,  0, 0,  0],
[0,  0, 0, -1],
[0,  0, 1,  0]])

In [81]:
Gamma_

Matrix([
[ 0, 1,  0, 0],
[-1, 0,  0, 0],
[ 0, 0,  0, 1],
[ 0, 0, -1, 0]])

In [86]:
eye(3)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [87]:
Gamma_ext=zeros(8,8)
Gamma_ext[2:4,2:4]=Gamma_[:2,:2]
Gamma_ext[6:8,6:8]=Gamma_[2:,2:]
Gamma_ext[:2,4:6]=eye(2)
Gamma_ext[4:6,:2]=-eye(2)

In [89]:
Gamma_dag_ext=zeros(8,8)
Gamma_dag_ext[:2,:2]=Gamma_dag[:2,:2]
Gamma_dag_ext[4:6,4:6]=Gamma_dag[2:,2:]
Gamma_dag_ext[2:4,6:8]=eye(2)
Gamma_dag_ext[6:8,2:4]=-eye(2)

In [88]:
Gamma_ext

Matrix([
[ 0,  0,  0, 0, 1, 0,  0, 0],
[ 0,  0,  0, 0, 0, 1,  0, 0],
[ 0,  0,  0, 1, 0, 0,  0, 0],
[ 0,  0, -1, 0, 0, 0,  0, 0],
[-1,  0,  0, 0, 0, 0,  0, 0],
[ 0, -1,  0, 0, 0, 0,  0, 0],
[ 0,  0,  0, 0, 0, 0,  0, 1],
[ 0,  0,  0, 0, 0, 0, -1, 0]])

In [90]:
Gamma_dag_ext

Matrix([
[0, -1,  0,  0, 0,  0, 0, 0],
[1,  0,  0,  0, 0,  0, 0, 0],
[0,  0,  0,  0, 0,  0, 1, 0],
[0,  0,  0,  0, 0,  0, 0, 1],
[0,  0,  0,  0, 0, -1, 0, 0],
[0,  0,  0,  0, 1,  0, 0, 0],
[0,  0, -1,  0, 0,  0, 0, 0],
[0,  0,  0, -1, 0,  0, 0, 0]])

In [92]:
Gamma_dag_Gamma=zeros(8,8)
Gamma_dag_Gamma[:2,:2]=Gamma_dag[:2,:2]
Gamma_dag_Gamma[4:6,4:6]=Gamma_dag[2:,2:]
Gamma_dag_Gamma[2:4,2:4]=Gamma_[:2,:2]
Gamma_dag_Gamma[6:8,6:8]=Gamma_[2:,2:]

In [93]:
Gamma_dag_Gamma

Matrix([
[0, -1,  0, 0, 0,  0,  0, 0],
[1,  0,  0, 0, 0,  0,  0, 0],
[0,  0,  0, 1, 0,  0,  0, 0],
[0,  0, -1, 0, 0,  0,  0, 0],
[0,  0,  0, 0, 0, -1,  0, 0],
[0,  0,  0, 0, 1,  0,  0, 0],
[0,  0,  0, 0, 0,  0,  0, 1],
[0,  0,  0, 0, 0,  0, -1, 0]])

In [36]:
Gamma = zeros(8, 8)
order=[eta[i] for i in range(4)] + [xi[i] for i in range(4)]
ij_list=[(i,j) for i in range(8) for j in range(i+1,8)]

# op_K=op_u_g.subs(gamma_xi)
# op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)*16
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

100%|██████████| 28/28 [07:10<00:00, 15.36s/it]


In [38]:
Gamma*4

Matrix([
[ 0, -1,  0,  1,  1,  0,  1,  0],
[ 1,  0, -1,  0,  0,  1,  0,  1],
[ 0,  1,  0, -1,  1,  0,  1,  0],
[-1,  0,  1,  0,  0,  1,  0,  1],
[-1,  0, -1,  0,  0, -1,  0,  1],
[ 0, -1,  0, -1,  1,  0, -1,  0],
[-1,  0, -1,  0,  0,  1,  0, -1],
[ 0, -1,  0, -1, -1,  0,  1,  0]])

In [205]:
Gamma*4

Matrix([
[ 0, -1,  0, -1,  1,  0, -1,  0],
[ 1,  0,  1,  0,  0,  1,  0, -1],
[ 0, -1,  0, -1, -1,  0,  1,  0],
[ 1,  0,  1,  0,  0, -1,  0,  1],
[-1,  0,  1,  0,  0,  1,  0,  1],
[ 0, -1,  0,  1, -1,  0, -1,  0],
[ 1,  0, -1,  0,  0,  1,  0,  1],
[ 0,  1,  0, -1, -1,  0, -1,  0]])

In [ ]:
Gamma = zeros(16, 16)
order=[eta[i] for i in range(8)] + [xi[i] for i in range(8)]
ij_list=[(i,j) for i in range(16) for j in range(i+1,16)]

# op_K=op_u_g.subs(gamma_xi)
# op_dm=recursiveapply(( op_K* rho_epr4 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)
for i,j in tqdm(ij_list):
        rs=I*order[i]*order[j]*( op_dm)
        matel=recursiveapply(rs.expand(),normalorder2).subs(constant_only)
        matel=matel.simplify()
        Gamma[i,j]=matel
        Gamma[j,i]=-matel

In [ ]:
Gamma

In [ ]:
op_dm=recursiveapply(( op_K* rho_epr2 * Dagger(op_K)).expand().subs(selfadjoint),normalorder2)


## Contraction

In [217]:
def get_Gamma(p01,p03,p12,p23):
    Gamma=zeros(8,8)
    Gamma[0,1]=p01
    Gamma[0,3]=-p03
    Gamma[1,2]=-p12
    Gamma[2,3]=p23

    Gamma[4:,4:]=-Gamma[:4,:4]

    Gamma[0,4]=Gamma[1,5]=Gamma[2,6]=Gamma[3,7]=1
    Gamma[0,6]=Gamma[1,7]=Gamma[2,4]=Gamma[3,5]=p01

    return Gamma-Gamma.T


In [221]:
G=get_Gamma(1,1,-1,1)/2

In [223]:
get_Gamma(1,1,-1,1)/2

Matrix([
[   0,  1/2,    0, -1/2,  1/2,    0,  1/2,    0],
[-1/2,    0,  1/2,    0,    0,  1/2,    0,  1/2],
[   0, -1/2,    0,  1/2,  1/2,    0,  1/2,    0],
[ 1/2,    0, -1/2,    0,    0,  1/2,    0,  1/2],
[-1/2,    0, -1/2,    0,    0, -1/2,    0,  1/2],
[   0, -1/2,    0, -1/2,  1/2,    0, -1/2,    0],
[-1/2,    0, -1/2,    0,    0,  1/2,    0, -1/2],
[   0, -1/2,    0, -1/2, -1/2,    0,  1/2,    0]])

In [224]:
get_Gamma(-1,1,-1,-1)/2

Matrix([
[   0, -1/2,    0, -1/2,  1/2,    0, -1/2,    0],
[ 1/2,    0,  1/2,    0,    0,  1/2,    0, -1/2],
[   0, -1/2,    0, -1/2, -1/2,    0,  1/2,    0],
[ 1/2,    0,  1/2,    0,    0, -1/2,    0,  1/2],
[-1/2,    0,  1/2,    0,    0,  1/2,    0,  1/2],
[   0, -1/2,    0,  1/2, -1/2,    0, -1/2,    0],
[ 1/2,    0, -1/2,    0,    0,  1/2,    0,  1/2],
[   0,  1/2,    0, -1/2, -1/2,    0, -1/2,    0]])

In [225]:
contraction_(get_Gamma(1,1,-1,1)/2, get_Gamma(-1,1,-1,-1)/2,4)

Matrix([
[0,  0, 0, -1,  0, 0,  0, 0],
[0,  0, 1,  0,  0, 0,  0, 0],
[0, -1, 0,  0,  0, 0,  0, 0],
[1,  0, 0,  0,  0, 0,  0, 0],
[0,  0, 0,  0,  0, 0,  0, 1],
[0,  0, 0,  0,  0, 0, -1, 0],
[0,  0, 0,  0,  0, 1,  0, 0],
[0,  0, 0,  0, -1, 0,  0, 0]])

In [78]:
contraction_(Gamma_dag, Gamma_,2)

Matrix([
[0, -1,  0, 0],
[1,  0,  0, 0],
[0,  0,  0, 1],
[0,  0, -1, 0]])

In [79]:
contraction_(Gamma_,Gamma_dag,2)

Matrix([
[ 0, 1, 0,  0],
[-1, 0, 0,  0],
[ 0, 0, 0, -1],
[ 0, 0, 1,  0]])

In [91]:
contraction_(Gamma_dag_ext,Gamma_ext,4)

Matrix([
[0, -1,  0, 0, 0,  0,  0, 0],
[1,  0,  0, 0, 0,  0,  0, 0],
[0,  0,  0, 1, 0,  0,  0, 0],
[0,  0, -1, 0, 0,  0,  0, 0],
[0,  0,  0, 0, 0, -1,  0, 0],
[0,  0,  0, 0, 1,  0,  0, 0],
[0,  0,  0, 0, 0,  0,  0, 1],
[0,  0,  0, 0, 0,  0, -1, 0]])

### Test whether the Gamma matrix for different subspace is just the direct sum

**Yes, it is just the simple direct sum of two covariance matrix**

In [ ]:
theta1 = Symbol('theta_1',real=True)
theta2 = Symbol('theta_2',real=True)


In [ ]:
Upsilon_=zeros(8,8)
Upsilon_[0,4]=Upsilon_[1,5]=cos(theta1)
Upsilon_[1,4]=sin(theta1)
Upsilon_[0,5]=-sin(theta1)
Upsilon_[2,6]=Upsilon_[3,7]=cos(theta2)
Upsilon_[3,6]=sin(theta2)
Upsilon_[2,7]=-sin(theta2)
Upsilon_=(Upsilon_-Upsilon_.T)

In [ ]:
Upsilon_

Matrix([
[            0,             0,             0,             0, cos(theta_1), -sin(theta_1),            0,             0],
[            0,             0,             0,             0, sin(theta_1),  cos(theta_1),            0,             0],
[            0,             0,             0,             0,            0,             0, cos(theta_2), -sin(theta_2)],
[            0,             0,             0,             0,            0,             0, sin(theta_2),  cos(theta_2)],
[-cos(theta_1), -sin(theta_1),             0,             0,            0,             0,            0,             0],
[ sin(theta_1), -cos(theta_1),             0,             0,            0,             0,            0,             0],
[            0,             0, -cos(theta_2), -sin(theta_2),            0,             0,            0,             0],
[            0,             0,  sin(theta_2), -cos(theta_2),            0,             0,            0,             0]])

In [ ]:
Upsilon_1=zeros(8,8)
Upsilon_1[0,4]=Upsilon_1[1,5]=cos(theta1)
Upsilon_1[1,4]=sin(theta1)
Upsilon_1[0,5]=-sin(theta1)
Upsilon_1[2,6]=Upsilon_1[3,7]=1
Upsilon_1=(Upsilon_1-Upsilon_1.T)

In [101]:
Upsilon_1

Matrix([
[            0,             0,  0,  0, cos(theta_1), -sin(theta_1), 0, 0],
[            0,             0,  0,  0, sin(theta_1),  cos(theta_1), 0, 0],
[            0,             0,  0,  0,            0,             0, 1, 0],
[            0,             0,  0,  0,            0,             0, 0, 1],
[-cos(theta_1), -sin(theta_1),  0,  0,            0,             0, 0, 0],
[ sin(theta_1), -cos(theta_1),  0,  0,            0,             0, 0, 0],
[            0,             0, -1,  0,            0,             0, 0, 0],
[            0,             0,  0, -1,            0,             0, 0, 0]])

In [105]:
Upsilon_2=zeros(8,8)
Upsilon_2[2,6]=Upsilon_2[3,7]=cos(theta2)
Upsilon_2[3,6]=sin(theta2)
Upsilon_2[2,7]=-sin(theta2)
Upsilon_2[0,4]=Upsilon_2[1,5]=1
Upsilon_2=(Upsilon_2-Upsilon_2.T)

In [106]:
Upsilon_2

Matrix([
[ 0,  0,             0,             0, 1, 0,            0,             0],
[ 0,  0,             0,             0, 0, 1,            0,             0],
[ 0,  0,             0,             0, 0, 0, cos(theta_2), -sin(theta_2)],
[ 0,  0,             0,             0, 0, 0, sin(theta_2),  cos(theta_2)],
[-1,  0,             0,             0, 0, 0,            0,             0],
[ 0, -1,             0,             0, 0, 0,            0,             0],
[ 0,  0, -cos(theta_2), -sin(theta_2), 0, 0,            0,             0],
[ 0,  0,  sin(theta_2), -cos(theta_2), 0, 0,            0,             0]])

In [107]:
contraction_(Upsilon_1,Upsilon_2,4) - Upsilon_

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]])

# correlated measurement

In [ ]:
a = Symbol('a',real=True)
b = Symbol('b',real=True)

: 

In [21]:
recursiveapply(((a + b * I*g[0]*g[1]) * (a- b* I*g[1]*g[2]) * (a- b* I*g[1]*g[2]) * (a + b * I*g[0]*g[1]) ).expand(),normalorder2)

a**4 + 2*I*a**3*b*\gamma_0*\gamma_1 - 2*I*a**3*b*\gamma_1*\gamma_2 + 2*a**2*b**2 + 2*I*a*b**3*\gamma_0*\gamma_1 + 2*I*a*b**3*\gamma_1*\gamma_2 + b**4

In [ ]:
recursiveapply(((a + b * I*g[0]*g[1]) * (a+ b* I*g[1]*g[2]) * (a+ b* I*g[1]*g[2]) * (a + b * I*g[0]*g[1]) ).expand(),normalorder2)

: 

In [19]:
recursiveapply(((a + -b * I*g[0]*g[1]) * (a+ -b* I*g[1]*g[2]) * (a+ -b* I*g[1]*g[2]) * (a + -b * I*g[0]*g[1]) ).expand(),normalorder2)

a**4 - 2*I*a**3*b*\gamma_0*\gamma_1 - 2*I*a**3*b*\gamma_1*\gamma_2 + 2*a**2*b**2 - 2*I*a*b**3*\gamma_0*\gamma_1 + 2*I*a*b**3*\gamma_1*\gamma_2 + b**4

In [20]:
recursiveapply(((a + b * I*g[0]*g[1]) * (a+ b* I*g[1]*g[2]) * (a+ b* I*g[1]*g[2]) * (a + b * I*g[0]*g[1]) ).expand(),normalorder2) + recursiveapply(((a + -b * I*g[0]*g[1]) * (a+ -b* I*g[1]*g[2]) * (a+ -b* I*g[1]*g[2]) * (a + -b * I*g[0]*g[1]) ).expand(),normalorder2)

2*a**4 + 4*a**2*b**2 + 2*b**4

# Obs-del

In [164]:
op_plus

1/2 - I*\gamma_0*\gamma_1/4 - I*\gamma_0*\gamma_3/4 + I*\gamma_1*\gamma_2/4 - I*\gamma_2*\gamma_3/4

In [167]:
op_pm

1/4 - I*\gamma_0*\gamma_1/4 - \gamma_0*\gamma_1*\gamma_2*\gamma_3/4 - I*\gamma_2*\gamma_3/4